# Otimização de hiperparâmetros


- Objetivo: Utilizar a otimização bayesiana de hiperparâmetros nos modelos de leves

# Pacotes

In [0]:
!pip install shap

In [0]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
from pyspark.sql.functions import *
from pyspark.sql.functions import expr
from pyspark.sql.types import ByteType,ShortType,LongType,IntegerType,DoubleType
import seaborn as sns
import shap
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE, SelectKBest, chi2
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge
from sklearn.model_selection import cross_val_score, train_test_split, StratifiedKFold
import statsmodels.api as sm

# Classes

In [0]:
%run /Users/felipe.espirito@bv.com.br/Modulos

# Funções

In [0]:
%run /Users/lucas.lima@bv.com.br/funcoes/funcoes_lucas

In [0]:
%run /Users/felipe.espirito@bv.com.br/funcoes

tabela_mmNtx(nomebase,nvar,nfxvar,ntarget) | última atualização: 16/03/2022

Out[4]: True

calc_perf(n_base,v_score,v_target) | última atualização: 03/03/2022

[1] "calcKS_R <- function(n_target,n_score)"

cria_fxs(nbase1,nbase2,nvar,n_fxs,nova_var) | última atualização 08/03/2022

calc_IV(nbase1,nvar_fx,ntarget) | última atualização 11/03/2022

aplicaRUPD(nbase,nvar) | Última atualização: 30/03/2022

encontra_modelo(X, y, classificador, n_divisoes, n_repeticoes,hiperparams = None, scoring="accuracy", random_state=1109) 
 Última atualização: 21/03/2022

# Dados

In [0]:
%sql 
refresh table cred_lucaslima.levesfast_escorada_cli;
select * from cred_lucaslima.levesfast_escorada_cli limit 10;

NuProposta,NuCpfCnpj,DtPrimeiroCadastro,QtMesesEmprego,DsEstadoCivil,CdUfCliente,CPF_main,Safra_main,P3_ult_ret,P4_BVS,SCORE_HRIM,SCORE_HSPN,SCORE_HVLD,QuodScore_v2,score_ajust,inad30,elegiveis,QuodScore,Blend_Novo_1_1_arred,RUPD,ScoreBacen,WoE_score_ajust,QtDiasAtrasoMinCtol1,QtDiasAtrasoMedCtol1,QtDiasAtrasoMaxCtol1,QtAtrasoAtual_Cto1,WoE_Int_Internas1,Int_Internas1,WoE_QtDiasAtrasoMaxCtol1_C,QtDiasAtrasoMaxCtol1_C,WoE_PP_PD_VPG01_PF_C,PP_PD_VPG01_PF_C,WoE_QtAtrasoMed_Cpf_C,QtAtrasoMed_Cpf_C,WoE_PP_PD_CPG01_PF_C,PP_PD_CPG01_PF_C,WoE_QtAtrasoMax_Cpf_C,QtAtrasoMax_Cpf_C,WoE_QtDiasAtrasoMinCtol1_C,QtDiasAtrasoMinCtol1_C,WoE_PP_HA_AMP01_PF_C,PP_HA_AMP01_PF,WoE_GC_AV_GRV01_PF_C,GC_AV_GRV01_PF,WoE_PF_CP_VOL04_PF_C,PF_CP_VOL04_PF_C,PF_CP_VOL04_PF,WoE_PP_AV_GRV04_PF_C,PP_AV_GRV04_PF,WoE_PP_HA_AMP02_PF_C,PP_HA_AMP02_PF,WoE_PF_CP_VOL02_PF_C,PF_CP_VOL02_PF,WoE_PF_CP_VOL01_PF_C,PF_CP_VOL01_PF,WoE_PP_PD_CPG02_PF_C,PP_PD_CPG02_PF,WoE_PF_CP_VOL03_PF_C,PF_CP_VOL03_PF,WoE_PP_AV_AMP02_PF_C,PP_AV_AMP02_PF,WoE_PP_HA_VLR02_PF_C,PP_HA_VLR02_PF,WoE_PP_AV_GRV02_PF_C,PP_AV_GRV02_PF,WoE_PF_PC_CPR02_PF_C,PF_PC_CPR02_PF,WoE_RM_PR_MOD04_PF_C,RM_PR_MOD04_PF,yhat1,yhat1_2,yhat1_3,yhat1_4,yhat1_5,logito_CRISP,logito_yhat1_4,yhat_blend,yhat_blend2,yhat_rf1_1,qsyhat1,qsyhat1_2,qsyhat1_3,qsyhat1_4,qsyhat1_5,qsyhat_rf1_1,qsyhat_blend,qsyhat_blend2,qsP3_ult_ret,qsP4_BVS,qsSCORE_HSPN,qsSCORE_HVLD,qsQuodScore_v2,qsQuodScore,qsBlend_Novo_1_1_arred,prob,prob90m12
371809887,59335114049,25/01/2005,148,CASADO,RS,59335114049,202109,677.0,546.0,674.0,674.0,987.0,766,796.0,0,1,751.0,990.0,8.5,981.0,0.302223,0.0,0.2222222222,2.0,-9.9999999E7,0.453493,1,0.420655,1,0.008917,2,0.313914,1,0.352094,1,-0.110795,3,0.114977,1,0.047749,46.21,-0.074026,22.25,0.377749,1,99.96,0.210222,0.0,0.036937,90.51,0.275167,99.89,0.030219,8.97,0.238348,97.8,0.311529,93.37,0.219652,0.0,0.022807,16.17,0.208731,0.0,0.249027,100.0,0.135206,91.67,983.1840971221311,987.9433124192875,991.0830497285433,989.0074081237948,982.3474417954187,4.59511985013459,4.499480241995284,990.1600405038525,991.094721653274,982.5330137952201,4.0,6.0,7.0,6.0,4.0,5,7.0,7.0,0.0,5.0,3.0,7.0,2,2.0,7.0,0.009839959496147466,0.03635736079591689
243497793,60391693972,07/05/2001,24,CASADO,PR,60391693972,202109,832.0,656.0,4.0,830.0,996.0,881,597.0,0,1,768.0,996.0,9.5,992.0,-0.147677,0.0,0.4375,5.0,-9.9999999E7,0.084284,2,-1.57E-4,2,-0.696138,4,-0.260612,3,0.352094,1,-0.505355,5,0.114977,1,-0.189917,70.39,0.274077,0.0,0.377749,1,99.15,0.210222,0.0,0.036937,63.51,0.275167,99.11,0.030219,15.09,0.238348,99.7,0.311529,98.78,0.219652,0.0,0.224565,0.0,0.208731,0.0,0.249027,100.0,0.135206,91.67,989.0249440212453,990.1658203016576,982.5076048122393,990.416961915669,976.5014933600704,5.517452896464707,4.6381313586029975,992.8123479592138,993.1949598429632,988.4579510693482,6.0,7.0,4.0,7.0,3.0,7,8.0,8.0,7.0,8.0,6.0,9.0,5,2.0,9.0,0.007187652040786197,0.02850861471780708
711284595,64023729787,07/10/2016,120,VIUVO,RJ,64023729787,202109,847.0,494.0,712.0,712.0,978.0,964,773.0,1,1,855.0,985.0,8.25,-9.9999999E7,0.174634,0.0,0.0,0.0,-9.9999999E7,0.453493,1,0.420655,1,0.008917,2,0.313914,1,0.043678,2,0.211916,2,0.114977,1,0.047749,24.49,0.274077,7.14,0.377749,1,99.76,0.210222,0.0,0.036937,50.05,0.275167,99.99,0.284493,34.22,0.238348,99.05,0.311529,99.94,0.219652,0.0,-0.358585,98.0,0.208731,0.0,0.249027,100.0,0.135206,91.67,984.5443116056454,986.3378526678829,985.9464288886153,988.4376189922115,991.3314676198484,4.1845914400698785,4.448368722316815,988.7773854761438,989.5393210725732,985.7252743308636,5.0,5.0,5.0,6.0,8.0,6,6.0,7.0,8.0,3.0,4.0,5.0,8,6.0,6.0,0.0112226145238562,0.040421146479068026
271595457,64970345953,07/01/2020,36,CASADO,SC,64970345953,202109,864.0,614.0,685.0,685.0,974.0,882,784.0,0,1,823.0,987.0,8.25,-9.9999999E7,0.302223,0.0,0.25,6.0,-9.9999999E7,0.084284,2,-1.57E-4,2,0.398254,1,0.313914,1,0.352094,1,0.211916,2,0.114977,1,0.396509,0.0,0.274077,0.0,-0.239079,3,304.0,0.210222,0.0,0.343908,0.0,-0.379273,0.0,-0.379273,0.0,-0.374953,3

In [0]:
base_cli=spark.sql('''
  select * from cred_lucaslima.levesfast_escorada_cli
  ''')
print('Base Clientes: ', base_cli.count())

base_cli_pd=base_cli.toPandas()

Base Clientes: 144330

In [0]:
base_cli_pd

,NuProposta,NuCpfCnpj,DtPrimeiroCadastro,QtMesesEmprego,DsEstadoCivil,CdUfCliente,CPF_main,Safra_main,P3_ult_ret,P4_BVS,SCORE_HRIM,SCORE_HSPN,SCORE_HVLD,QuodScore_v2,score_ajust,inad30,elegiveis,QuodScore,Blend_Novo_1_1_arred,RUPD,ScoreBacen,WoE_score_ajust,QtDiasAtrasoMinCtol1,QtDiasAtrasoMedCtol1,QtDiasAtrasoMaxCtol1,QtAtrasoAtual_Cto1,WoE_Int_Internas1,Int_Internas1,WoE_QtDiasAtrasoMaxCtol1_C,QtDiasAtrasoMaxCtol1_C,WoE_PP_PD_VPG01_PF_C,PP_PD_VPG01_PF_C,WoE_QtAtrasoMed_Cpf_C,QtAtrasoMed_Cpf_C,WoE_PP_PD_CPG01_PF_C,PP_PD_CPG01_PF_C,WoE_QtAtrasoMax_Cpf_C,QtAtrasoMax_Cpf_C,WoE_QtDiasAtrasoMinCtol1_C,QtDiasAtrasoMinCtol1_C,WoE_PP_HA_AMP01_PF_C,PP_HA_AMP01_PF,WoE_GC_AV_GRV01_PF_C,GC_AV_GRV01_PF,WoE_PF_CP_VOL04_PF_C,PF_CP_VOL04_PF_C,PF_CP_VOL04_PF,WoE_PP_AV_GRV04_PF_C,PP_AV_GRV04_PF,WoE_PP_HA_AMP02_PF_C,PP_HA_AMP02_PF,WoE_PF_CP_VOL02_PF_C,PF_CP_VOL02_PF,WoE_PF_CP_VOL01_PF_C,PF_CP_VOL01_PF,WoE_PP_PD_CPG02_PF_C,PP_PD_CPG02_PF,WoE_PF_CP_VOL03_PF_C,PF_CP_VOL03_PF,WoE_PP_AV_AMP02_PF_C,PP_AV_AMP02_PF,WoE_PP_HA_VLR02_PF_C,PP_HA_VLR02_PF,WoE_PP_AV_GRV02_PF_C,PP_AV_GRV02_PF,WoE_PF_PC_CPR02_PF_C,PF_PC_CPR02_PF,WoE_RM_PR_MOD04_PF_C,RM_PR_MOD04_PF,yhat1,yhat1_2,yhat1_3,yhat1_4,yhat1_5,logito_CRISP,logito_yhat1_4,yhat_blend,yhat_blend2,yhat_rf1_1,qsyhat1,qsyhat1_2,qsyhat1_3,qsyhat1_4,qsyhat1_5,qsyhat_rf1_1,qsyhat_blend,qsyhat_blend2,qsP3_ult_ret,qsP4_BVS,qsSCORE_HSPN,qsSCORE_HVLD,qsQuodScore_v2,qsQuodScore,qsBlend_Novo_1_1_arred,prob,prob90m12
0,371809887,59335114049,25/01/2005,148,CASADO,RS,59335114049,202109,677.0,546.0,674.0,674.0,987.0,766,796.0,0,1,751.0,990.0,8.50,981.0,0.302223,0.0,2.222222e-01,2.0,-99999999.0,0.453493,1,0.420655,1,0.008917,2,0.313914,1,0.352094,1,-0.110795,3,0.114977,1,0.047749,46.21,-0.074026,22.25,0.377749,1,99.96,0.210222,0.00,0.036937,90.51,0.275167,99.89,0.030219,8.97,0.238348,97.80,0.311529,93.37,0.219652,0.00,0.022807,16.17,0.208731,0.00,0.249027,100.00,0.135206,91.67,983.184097,987.943312,991.083050,989.007408,982.347442,4.595120,4.499480,990.160041,991.094722,982.533014,4.0,6.0,7.0,6.0,4.0,5,7.0,7.0,0.0,5.0,3.0,7.0,2,2.0,7.0,0.009840,0.036357
1,243497793,60391693972,07/05/2001,24,CASADO,PR,60391693972,202109,832.0,656.0,4.0,830.0,996.0,881,597.0,0,1,768.0,996.0,9.50,992.0,-0.147677,0.0,4.375000e-01,5.0,-99999999.0,0.084284,2,-0.000157,2,-0.696138,4,-0.260612,3,0.352094,1,-0.505355,5,0.114977,1,-0.189917,70.39,0.274077,0.00,0.377749,1,99.15,0.210222,0.00,0.036937,63.51,0.275167,99.11,0.030219,15.09,0.238348,99.70,0.311529,98.78,0.219652,0.00,0.224565,0.00,0.208731,0.00,0.249027,100.00,0.135206,91.67,989.024944,990.165820,982.507605,990.416962,976.501493,5.517453,4.638131,992.812348,993.194960,988.457951,6.0,7.0,4.0,7.0,3.0,7,8.0,8.0,7.0,8.0,6.0,9.0,5,2.0,9.0,0.007188,0.028509
2,711284595,64023729787,07/10/2016,120,VIUVO,RJ,64023729787,202109,847.0,494.0,712.0,712.0,978.0,964,773.0,1,1,855.0,985.0,8.25,-99999999.0,0.174634,0.0,0.000000e+00,0.0,-99999999.0,0.453493,1,0.420655,1,0.008917,2,0.313914,1,0.043678,2,0.211916,2,0.114977,1,0.047749,24.49,0.274077,7.14,0.377749,1,99.76,0.210222,0.00,0.036937,50.05,0.275167,99.99,0.284493,34.22,0.238348,99.05,0.311529,99.94,0.219652,0.00,-0.358585,98.00,0.208731,0.00,0.249027,100.00,0.135206,91.67,984.544312,986.337853,985.946429,988.437619,991.331468,4.184591,4.448369,988.777385,989.539321,985.725274,5.0,5.0,5.0,6.0,8.0,6,6.0,7.0,8.0,3.0,4.0,5.0,8,6.0,6.0,0.011223,0.040421
3,271595457,64970345953,07/01/2020,36,CASADO,SC,64970345953,202109,864.0,614.0,685.0,685.0,974.0,882,784.0,0,1,823.0,987.0,8.25,-99999999.0,0.302223,0.0,2.500000e-01,6.0,-99999999.0,0.084284,2,-0.000157,2,0.398254,1,0.313914,1,0.352094,1,0.211916,2,0.114977,1,0.396509,0.00,0.274077,0.00,-0.239079,3,304.00,0.210222,0.00,0.343908,0.00,-0.379273,0.00,-0.379273,0.00,-0.374953,304.00,-0.246304,304.00,0.219652,0.00,0.224565,304.00,0.208731,0.00,-0.038755,74.02,0.280347,304.00,985.921829,986.778597,984.665089,985.691144,986.824230,4.329721,4.232464,987.212979,987.300838,985.127323,5.0,6.0,5.0,5.0,6.0,6,6.0,6.0,

# Conversões

In [0]:
base_cli_pd["Safra_main"] = pd.to_datetime(base_cli_pd["Safra_main"].astype(str), format='%Y%m')

In [0]:
base_cli_cat=base_cli_pd[(base_cli_pd.Safra_main < "2021-11") & (base_cli_pd.elegiveis==1)]
base_cli_cat=base_cli_cat.fillna(-99999999)
base_cli_cat.head()

,NuProposta,NuCpfCnpj,DtPrimeiroCadastro,QtMesesEmprego,DsEstadoCivil,CdUfCliente,CPF_main,Safra_main,P3_ult_ret,P4_BVS,SCORE_HRIM,SCORE_HSPN,SCORE_HVLD,QuodScore_v2,score_ajust,inad30,elegiveis,QuodScore,Blend_Novo_1_1_arred,RUPD,ScoreBacen,WoE_score_ajust,QtDiasAtrasoMinCtol1,QtDiasAtrasoMedCtol1,QtDiasAtrasoMaxCtol1,QtAtrasoAtual_Cto1,WoE_Int_Internas1,Int_Internas1,WoE_QtDiasAtrasoMaxCtol1_C,QtDiasAtrasoMaxCtol1_C,WoE_PP_PD_VPG01_PF_C,PP_PD_VPG01_PF_C,WoE_QtAtrasoMed_Cpf_C,QtAtrasoMed_Cpf_C,WoE_PP_PD_CPG01_PF_C,PP_PD_CPG01_PF_C,WoE_QtAtrasoMax_Cpf_C,QtAtrasoMax_Cpf_C,WoE_QtDiasAtrasoMinCtol1_C,QtDiasAtrasoMinCtol1_C,WoE_PP_HA_AMP01_PF_C,PP_HA_AMP01_PF,WoE_GC_AV_GRV01_PF_C,GC_AV_GRV01_PF,WoE_PF_CP_VOL04_PF_C,PF_CP_VOL04_PF_C,PF_CP_VOL04_PF,WoE_PP_AV_GRV04_PF_C,PP_AV_GRV04_PF,WoE_PP_HA_AMP02_PF_C,PP_HA_AMP02_PF,WoE_PF_CP_VOL02_PF_C,PF_CP_VOL02_PF,WoE_PF_CP_VOL01_PF_C,PF_CP_VOL01_PF,WoE_PP_PD_CPG02_PF_C,PP_PD_CPG02_PF,WoE_PF_CP_VOL03_PF_C,PF_CP_VOL03_PF,WoE_PP_AV_AMP02_PF_C,PP_AV_AMP02_PF,WoE_PP_HA_VLR02_PF_C,PP_HA_VLR02_PF,WoE_PP_AV_GRV02_PF_C,PP_AV_GRV02_PF,WoE_PF_PC_CPR02_PF_C,PF_PC_CPR02_PF,WoE_RM_PR_MOD04_PF_C,RM_PR_MOD04_PF,yhat1,yhat1_2,yhat1_3,yhat1_4,yhat1_5,logito_CRISP,logito_yhat1_4,yhat_blend,yhat_blend2,yhat_rf1_1,qsyhat1,qsyhat1_2,qsyhat1_3,qsyhat1_4,qsyhat1_5,qsyhat_rf1_1,qsyhat_blend,qsyhat_blend2,qsP3_ult_ret,qsP4_BVS,qsSCORE_HSPN,qsSCORE_HVLD,qsQuodScore_v2,qsQuodScore,qsBlend_Novo_1_1_arred,prob,prob90m12
0,371809887,59335114049,25/01/2005,148,CASADO,RS,59335114049,2021-09-01,677.0,546.0,674.0,674.0,987.0,766,796.0,0,1,751.0,990.0,8.50,981.0,0.302223,0.0,0.222222,2.0,-99999999.0,0.453493,1,0.420655,1,0.008917,2,0.313914,1,0.352094,1,-0.110795,3,0.114977,1,0.047749,46.21,-0.074026,22.25,0.377749,1,99.96,0.210222,0.00,0.036937,90.51,0.275167,99.89,0.030219,8.97,0.238348,97.80,0.311529,93.37,0.219652,0.00,0.022807,16.17,0.208731,0.00,0.249027,100.00,0.135206,91.67,983.184097,987.943312,991.083050,989.007408,982.347442,4.595120,4.499480,990.160041,991.094722,982.533014,4.0,6.0,7.0,6.0,4.0,5,7.0,7.0,0.0,5.0,3.0,7.0,2,2.0,7.0,0.009840,0.036357
1,243497793,60391693972,07/05/2001,24,CASADO,PR,60391693972,2021-09-01,832.0,656.0,4.0,830.0,996.0,881,597.0,0,1,768.0,996.0,9.50,992.0,-0.147677,0.0,0.437500,5.0,-99999999.0,0.084284,2,-0.000157,2,-0.696138,4,-0.260612,3,0.352094,1,-0.505355,5,0.114977,1,-0.189917,70.39,0.274077,0.00,0.377749,1,99.15,0.210222,0.00,0.036937,63.51,0.275167,99.11,0.030219,15.09,0.238348,99.70,0.311529,98.78,0.219652,0.00,0.224565,0.00,0.208731,0.00,0.249027,100.00,0.135206,91.67,989.024944,990.165820,982.507605,990.416962,976.501493,5.517453,4.638131,992.812348,993.194960,988.457951,6.0,7.0,4.0,7.0,3.0,7,8.0,8.0,7.0,8.0,6.0,9.0,5,2.0,9.0,0.007188,0.028509
2,711284595,64023729787,07/10/2016,120,VIUVO,RJ,64023729787,2021-09-01,847.0,494.0,712.0,712.0,978.0,964,773.0,1,1,855.0,985.0,8.25,-99999999.0,0.174634,0.0,0.000000,0.0,-99999999.0,0.453493,1,0.420655,1,0.008917,2,0.313914,1,0.043678,2,0.211916,2,0.114977,1,0.047749,24.49,0.274077,7.14,0.377749,1,99.76,0.210222,0.00,0.036937,50.05,0.275167,99.99,0.284493,34.22,0.238348,99.05,0.311529,99.94,0.219652,0.00,-0.358585,98.00,0.208731,0.00,0.249027,100.00,0.135206,91.67,984.544312,986.337853,985.946429,988.437619,991.331468,4.184591,4.448369,988.777385,989.539321,985.725274,5.0,5.0,5.0,6.0,8.0,6,6.0,7.0,8.0,3.0,4.0,5.0,8,6.0,6.0,0.011223,0.040421
3,271595457,64970345953,07/01/2020,36,CASADO,SC,64970345953,2021-09-01,864.0,614.0,685.0,685.0,974.0,882,784.0,0,1,823.0,987.0,8.25,-99999999.0,0.302223,0.0,0.250000,6.0,-99999999.0,0.084284,2,-0.000157,2,0.398254,1,0.313914,1,0.352094,1,0.211916,2,0.114977,1,0.396509,0.00,0.274077,0.00,-0.239079,3,304.00,0.210222,0.00,0.343908,0.00,-0.379273,0.00,-0.379273,0.00,-0.374953,304.00,-0.246304,304.00,0.219652,0.00,0.224565,304.00,0.208731,0.00,-0.038755,74.02,0.280347,304.00,985.921829,986.778597,984.665089,985.691144,986.824230,4.329721,4.232464,987.212979,987.300838,985.127323,5.0,6.0,5.0,5.0,6.0,6,6.0,6.0,

In [0]:
vars_WoE=list(base_cli_cat.filter(like='WoE').columns)

X_list=vars_WoE+['P3_ult_ret','P4_BVS','SCORE_HSPN','QuodScore_v2','QuodScore']
X_list

# Lista com variáveis contínuas, com exceção das da B3, que são categóricas.
lista_b3_woe=['WoE_mpv_score','WoE_pb_qt_propavgdiaauto_pv',
              'WoE_pb_qt_diasfina_max_pv','WoE_pb_td_primcompr_pv']

# variáveis adicionais
X_list3 = X_list 

y=base_cli_cat.inad30

X1=base_cli_cat[X_list]
X3=base_cli_cat[X_list3]
X_geral = base_cli_cat

# Otimização Clientes

In [0]:
X=X_geral

prefixos = []
safras_dev = [safra for safra in base_cli_pd.Safra_main.unique() if safra < pd.to_datetime("2021-11")]
safras_teste = [safra for safra in base_cli_pd.Safra_main.unique() if safra > pd.to_datetime("2021-11") and safra < pd.to_datetime("2022-01")]
safras_nov = [pd.to_datetime("2021-11")]

variaveis_PF = [col for col in base_cli_pd.columns if "WoE_PF" in col]
variaveis_atraso = [col for col in base_cli_pd.columns if "WoE_Qt" in col]
variaveis_atraso_cont  = [col for col in base_cli_pd.columns if "WoE" not in col and "Qt" in col]
variaveis_continuas = [col for col in base_cli_pd.columns if "WoE" not in col and "_PF" in col]

restrito = [
 'WoE_score_ajust',
 'WoE_PF_CP_VOL04_PF_C',
 'P4_BVS',
 'QuodScore_v2',
 'SCORE_HSPN'
]

aberto = [
  *variaveis_PF,
  *variaveis_atraso,
 'WoE_score_ajust',
 'P4_BVS',
 'QuodScore_v2',
 'SCORE_HSPN'
]

aberto_continuo = [
  *variaveis_continuas,
  *variaveis_atraso_cont,
  'score_ajust',
  'P4_BVS',
  'QuodScore_v2',
  'SCORE_HSPN'
]

X_restrito=X[restrito]
X_aberto=X[aberto]
X_aberto_continuo=X[aberto_continuo]

vis = Visualizador(base=base_cli_pd, base_cat=base_cli_cat)
random_state = 1505 # seed geral

## Bayesiana

### AdaBoost

#### Restrito

In [0]:
prefixo = "ab_restrito"
prefixos.append(prefixo)

otimizador = OtimizadorBayesiano(classificador="adaboost", n_estimadores = 100, random_state=random_state, max_avaliacoes=100)
melhores_params, tentativas = otimizador.otimiza_hiperparams(X_restrito, y, tam_teste=.3)

0%| | 0/100 [00:00<?, ?trial/s, best loss=?] 2%|▏ | 2/100 [00:19<15:44, 9.64s/trial, best loss: -0.6960992051103801] 4%|▍ | 4/100 [00:23<08:14, 5.15s/trial, best loss: -0.6995952764918482] 6%|▌ | 6/100 [00:27<05:49, 3.72s/trial, best loss: -0.700286435072536]  8%|▊ | 8/100 [00:29<04:03, 2.65s/trial, best loss: -0.700286435072536] 9%|▉ | 9/100 [00:30<03:28, 2.29s/trial, best loss: -0.700286435072536] 10%|█ | 10/100 [00:32<03:20, 2.22s/trial, best loss: -0.700286435072536] 11%|█ | 11/100 [00:33<02:50, 1.91s/trial, best loss: -0.70086604479556]  13%|█▎ | 13/100 [00:35<02:13, 1.53s/trial, best loss: -0.7010067704016251] 14%|█▍ | 14/100 [00:36<02:01, 1.41s/trial, best loss: -0.7010067704016251] 16%|█▌ | 16/100 [00:39<02:01, 1.45s/trial, best loss: -0.7010067704016251] 18%|█▊ | 18/100 [00:40<01:30, 1.11s/trial, best loss: -0.7010067704016251] 19%|█▉ | 19/100 [00:41<01:28, 1.09s/trial, best loss: -0.7010067704016251] 20%|██ | 20/100 [00:42<01:25, 1.07s/trial, best loss: -0.7010067704016251] 21%|██ | 21/100 [00:44<01:42, 1.30s/trial, best loss: -0.7010067704016251] 22%|██▏ | 22/100 [00:46<02:02, 1.56s/trial, best loss: -0.7010067704016251] 23%|██▎ | 23/100 [00:47<01:48, 1.41s/trial, best loss: -0.7010067704016251] 25%|██▌ | 25/100 [00:49<01:32, 1.24s/trial, best loss: -0.7010067704016251] 27%|██▋ | 27/100 [00:51<01:23, 1.15s/trial, best loss: -0.7010067704016251] 30%|███ | 30/100 [00:52<00:54, 1.28trial/s, best loss: -0.7010067704016251] 32%|███▏ | 32/100 [00:53<00:47, 1.43trial/s, best loss: -0.7010067704016251] 38%|███▊ | 38/100 [00:54<00:24, 2.50trial/s, best loss: -0.7010067704016251] 39%|███▉ | 39/100 [00:55<00:28, 2.12trial/s, best loss: -0.7010067704016251] 40%|████ | 40/100 [00:57<00:41, 1.44trial/s, best loss: -0.7014809634142036] 41%|████ | 41/100 [00:59<00:54, 1.09trial/s, best loss: -0.7014809634142036] 43%|████▎ | 43/100 [01:03<01:12, 1.28s/trial, best loss: -0.7014809634142036] 45%|████▌ | 45/100 [01:04<00:56, 1.03s/trial, best loss: -0.7014809634142036] 46%|████▌ | 46/100 [01:06<01:05, 1.22s/trial, best loss: -0.7014809634142036] 48%|████▊ | 48/100 [01:07<00:50, 1.03trial/s, best loss: -0.7014809634142036] 49%|████▉ | 49/100 [01:08<00:49, 1.02trial/s, best loss: -0.7014809634142036] 50%|█████ | 50/100 [01:09<00:49, 1.02trial/s, best loss: -0.7014809634142036] 53%|█████▎ | 53/100 [01:10<00:31, 1.49trial/s, best loss: -0.7014809634142036] 54%|█████▍ | 54/100 [01:13<00:42, 1.08trial/s, best loss: -0.7014809634142036] 55%|█████▌ | 55/100 [01:14<00:42, 1.06trial/s, best loss: -0.7014809634142036] 56%|█████▌ | 56/100 [01:16<00:52, 1.19s/trial, best loss: -0.7014809634142036] 57%|█████▋ | 57/100 [01:17<00:49, 1.15s/trial, best loss: -0.7014809634142036] 58%|█████▊ | 58/100 [01:18<00:46, 1.11s/trial, best loss: -0.7014809634142036] 60%|██████ | 60/100 [01:24<01:17, 1.93s/trial, best loss: -0.7014809634142036] 61%|██████ | 61/100 [01:26<01:16, 1.96s/trial, best loss: -0.7014809634142036] 62%|██████▏ | 62/100 [01:28<01:14, 1.97s/trial, best loss: -0.7014809634142036] 64%|██████▍ | 64/100 [01:29<00:48, 1.36s/trial, best loss: -0.7014809634142036] 66%|██████▌ | 66/100 [01:33<00:54, 1.61s/trial, best loss: -0.7014809634142036] 67%|██████▋ | 67/100 [01:38<01:16, 2.33s/trial, best loss: -0.7014809634142036] 69%|██████▉ | 69/100 [01:41<01:02, 2.02s/trial, best loss: -0.7014809634142036] 71%|███████ | 71/100 [01:44<00:53, 1.84s/trial, best loss: -0.7014809634142036] 72%|███████▏ | 72/100 [01:45<00:46, 1.67s/trial, best loss: -0.7014809634142036] 73%|███████▎ | 73/100 [01:47<00:47, 1.75s/trial, best loss: -0.7014809634142036] 75%|███████▌ | 75/100 [01:48<00:31, 1.26s/trial, best loss: -0.7014809634142036] 76%|███████▌ | 76/100 [01:55<01:00, 2.51s/trial, best loss: -0.7014809634142036] 78%|███████▊ | 78/100 [02:03<01:07, 3.08s/trial, best loss: -0.7014809634142036] 79%|███████▉ | 79/100 [02:05<00:59, 2.85s/trial, best loss: -0.7014809634142036] 81%|████████ | 81/100 [02:08<00:44, 2.34s/tr

In [0]:
modelo_ab = otimizador.modelo
target = 'inad30'
X_train, X_test, y_train, y_test = train_test_split(X_restrito, y, train_size=.7, random_state=random_state)

temp_train=X_train.join(y_train,rsuffix='r')
temp_train[f'yhat_{prefixo}']=modelo_ab.predict_proba(X_train[restrito])[:,1]
print("KS treino: " , calc_perf(temp_train,f'yhat_{prefixo}', target))
  
temp_test=X_test.join(y_test,rsuffix='r')
temp_test[f'yhat_{prefixo}']=modelo_ab.predict_proba(temp_test[restrito])[:,1]
print("KS teste: " , calc_perf(temp_test,f'yhat_{prefixo}', target))

KS treino: (31.832084009075157, 53218, 2.3356758991318727, 51975, 1243)
KS teste: (27.08699776034893, 22809, 2.24472795826209, 22297, 512)

In [0]:
cols = restrito
modelos = [prefixo, *cols]

tab_KS = calcula_KS(modelo_ab, X_train, X_test, y_train, y_test, target, nscores=cols, restrito=restrito, prefixo=prefixo, safras=safras_dev, base_pd=base_cli_pd, base_cat=base_cli_cat)
tab_KS

vis.tabela_qs(modelos, "Safra_main", safras_dev)
vis.tabela_qs(modelos, "Safra_main", safras_nov)

col_0,ab_restrito,WoE_score_ajust,WoE_PF_CP_VOL04_PF_C,P4_BVS,QuodScore_v2,SCORE_HSPN
qs_ab_restrito,,,,,,
0,3.898622,2.467365,2.514751,4.560485,4.021724,1.250390
1,2.729288,1.924554,1.718287,2.985075,3.066038,3.610406
2,2.355549,1.584174,1.327765,2.387195,2.391350,2.975697
3,2.022021,1.427862,NaN,1.965824,2.005402,2.598036
4,1.671174,1.252926,NaN,1.606453,1.687909,1.943281
5,1.828761,NaN,NaN,1.281348,1.505525,1.670426
6,1.621846,NaN,NaN,1.362398,1.270884,1.452768
7,1.143372,NaN,NaN,0.976482,0.927411,1.105756
8,0.944502,NaN,NaN,0.779823,1.009891,0.694591


#### Aberto

In [0]:
prefixo = "ab_aberto"
prefixos.append(prefixo)

otimizador = OtimizadorBayesiano(classificador="adaboost", n_estimadores = 100, random_state=random_state, max_avaliacoes=100)
melhores_params, tentativas = otimizador.otimiza_hiperparams(X_aberto, y, tam_teste=.3)

0%| | 0/100 [00:00<?, ?trial/s, best loss=?] 1%| | 1/100 [00:17<28:11, 17.08s/trial, best loss: -0.7176675825301051] 2%|▏ | 2/100 [00:18<12:27, 7.62s/trial, best loss: -0.7176675825301051] 3%|▎ | 3/100 [00:20<08:10, 5.06s/trial, best loss: -0.7238736990891116] 4%|▍ | 4/100 [00:23<06:48, 4.26s/trial, best loss: -0.7238736990891116] 6%|▌ | 6/100 [00:28<05:13, 3.34s/trial, best loss: -0.7238736990891116] 9%|▉ | 9/100 [00:34<03:59, 2.63s/trial, best loss: -0.7238736990891116] 10%|█ | 10/100 [00:36<03:45, 2.51s/trial, best loss: -0.7238736990891116] 11%|█ | 11/100 [00:38<03:33, 2.40s/trial, best loss: -0.7238736990891116] 12%|█▏ | 12/100 [00:39<03:01, 2.06s/trial, best loss: -0.7238736990891116] 15%|█▌ | 15/100 [00:40<01:40, 1.18s/trial, best loss: -0.7238736990891116] 16%|█▌ | 16/100 [00:41<01:36, 1.15s/trial, best loss: -0.7238736990891116] 17%|█▋ | 17/100 [00:44<02:09, 1.56s/trial, best loss: -0.7238736990891116] 18%|█▊ | 18/100 [00:47<02:38, 1.94s/trial, best loss: -0.7238736990891116] 21%|██ | 21/100 [00:49<01:42, 1.30s/trial, best loss: -0.7238736990891116] 22%|██▏ | 22/100 [00:50<01:36, 1.24s/trial, best loss: -0.7238736990891116] 23%|██▎ | 23/100 [00:51<01:31, 1.19s/trial, best loss: -0.7238736990891116] 27%|██▋ | 27/100 [00:52<00:49, 1.49trial/s, best loss: -0.7242687329334206] 29%|██▉ | 29/100 [00:53<00:44, 1.60trial/s, best loss: -0.7242687329334206] 30%|███ | 30/100 [00:54<00:48, 1.45trial/s, best loss: -0.7242687329334206] 34%|███▍ | 34/100 [00:56<00:39, 1.67trial/s, best loss: -0.7248443876796126] 35%|███▌ | 35/100 [00:57<00:42, 1.52trial/s, best loss: -0.7248443876796126] 36%|███▌ | 36/100 [01:00<01:08, 1.07s/trial, best loss: -0.7248443876796126] 38%|███▊ | 38/100 [01:01<00:54, 1.13trial/s, best loss: -0.7248443876796126] 39%|███▉ | 39/100 [01:03<01:07, 1.10s/trial, best loss: -0.7248443876796126] 40%|████ | 40/100 [01:06<01:30, 1.52s/trial, best loss: -0.7248443876796126] 42%|████▏ | 42/100 [01:07<01:05, 1.13s/trial, best loss: -0.7248443876796126] 43%|████▎ | 43/100 [01:08<01:02, 1.10s/trial, best loss: -0.7248443876796126] 45%|████▌ | 45/100 [01:10<00:58, 1.07s/trial, best loss: -0.7248443876796126] 47%|████▋ | 47/100 [01:11<00:46, 1.15trial/s, best loss: -0.7248443876796126] 49%|████▉ | 49/100 [01:12<00:38, 1.34trial/s, best loss: -0.7248443876796126] 50%|█████ | 50/100 [01:16<01:09, 1.38s/trial, best loss: -0.7248443876796126] 51%|█████ | 51/100 [01:19<01:25, 1.74s/trial, best loss: -0.7248443876796126] 55%|█████▌ | 55/100 [01:22<00:53, 1.19s/trial, best loss: -0.7251390183777122] 57%|█████▋ | 57/100 [01:23<00:42, 1.00trial/s, best loss: -0.7251390183777122] 60%|██████ | 60/100 [01:24<00:29, 1.34trial/s, best loss: -0.7251390183777122] 61%|██████ | 61/100 [01:25<00:30, 1.27trial/s, best loss: -0.7251390183777122] 62%|██████▏ | 62/100 [01:27<00:38, 1.01s/trial, best loss: -0.7251390183777122] 65%|██████▌ | 65/100 [01:30<00:35, 1.01s/trial, best loss: -0.7252067097397404] 66%|██████▌ | 66/100 [01:31<00:34, 1.01s/trial, best loss: -0.7252067097397404] 67%|██████▋ | 67/100 [01:32<00:33, 1.01s/trial, best loss: -0.7252067097397404] 70%|███████ | 70/100 [01:33<00:21, 1.43trial/s, best loss: -0.7252067097397404] 71%|███████ | 71/100 [01:34<00:21, 1.32trial/s, best loss: -0.7252067097397404] 74%|███████▍ | 74/100 [01:37<00:22, 1.16trial/s, best loss: -0.7252067097397404] 77%|███████▋ | 77/100 [01:40<00:21, 1.09trial/s, best loss: -0.7252067097397404] 79%|███████▉ | 79/100 [01:42<00:19, 1.06trial/s, best loss: -0.7252067097397404] 80%|████████ | 80/100 [01:43<00:19, 1.05trial/s, best loss: -0.7252067097397404] 83%|████████▎ | 83/100 [01:46<00:16, 1.03trial/s, best loss: -0.7252067097397404] 85%|████████▌ | 85/100 [01:47<00:12, 1.19trial/s, best loss: -0.7252067097397404] 86%|████████▌ | 86/100 [01:48<00:12, 1.15trial/s, best loss: -0.7252067097397404] 89%|████████▉ | 89/100 [01:52<00:11, 1.06s/trial, best loss: -0.7252067097397404] 90%|█████████ | 90/100 [01:55<00:13, 1

In [0]:
modelo_ab_aberto = otimizador.modelo
target = 'inad30'
X_train, X_test, y_train, y_test = train_test_split(X_aberto, y, train_size=.7, random_state=random_state)

temp_train=X_train.join(y_train,rsuffix='r')
temp_train[f'yhat_{prefixo}']=modelo_ab_aberto.predict_proba(X_train[aberto])[:,1]
print("KS treino: " , calc_perf(temp_train,f'yhat_{prefixo}', target))
  
temp_test=X_test.join(y_test,rsuffix='r')
temp_test[f'yhat_{prefixo}']=modelo_ab_aberto.predict_proba(temp_test[aberto])[:,1]
print("KS teste: " , calc_perf(temp_test,f'yhat_{prefixo}', target))

KS treino: (35.96485716839699, 53218, 2.3356758991318727, 51975, 1243)
KS teste: (32.996521393012515, 22809, 2.24472795826209, 22297, 512)

In [0]:
cols = aberto
modelos = [prefixo, *cols]

tab_KS = calcula_KS(modelo_ab_aberto, X_train, X_test, y_train, y_test, target, nscores=cols, aberto=aberto, prefixo=prefixo, safras=safras_dev, base_pd=base_cli_pd, base_cat=base_cli_cat)
tab_KS

vis.tabela_qs(modelos, "Safra_main", safras_dev)
vis.tabela_qs(modelos, "Safra_main", safras_nov)

col_0,ab_aberto,WoE_PF_CP_VOL04_PF_C,WoE_PF_CP_VOL02_PF_C,WoE_PF_CP_VOL01_PF_C,WoE_PF_CP_VOL03_PF_C,WoE_PF_PC_CPR02_PF_C,WoE_QtDiasAtrasoMaxCtol1_C,WoE_QtAtrasoMed_Cpf_C,WoE_QtAtrasoMax_Cpf_C,WoE_QtDiasAtrasoMinCtol1_C,WoE_score_ajust,P4_BVS,QuodScore_v2,SCORE_HSPN
qs_ab_aberto,,,,,,,,,,,,,,
0,3.997783,2.514751,2.834066,3.240335,2.561404,2.826150,2.549270,2.919095,2.932939,1.858934,2.467365,4.560485,4.021724,1.250390
1,3.533569,1.718287,1.969660,2.254973,1.903100,1.930585,1.214792,1.950833,2.074496,NaN,1.924554,2.985075,3.066038,3.610406
2,2.326548,1.327765,1.432791,1.269113,1.388214,1.476533,NaN,1.359164,1.547718,NaN,1.584174,2.387195,2.391350,2.975697
3,2.052135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.407948,NaN,1.427862,1.965824,2.005402,2.598036
4,1.704309,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.252926,1.606453,1.687909,1.943281
5,1.683758,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.281348,1.505525,1.670426
6,1.316428,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.362398,1.270884,1.452768
7,1.025357,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.976482,0.927411,1.105756
8,0.650970,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.779823,1.009891,0.694591


#### Aberto Contínuo

In [0]:
prefixo = "ab_aberto_continuo"
prefixos.append(prefixo)

otimizador = OtimizadorBayesiano(classificador="adaboost", n_estimadores = 100, random_state=random_state, max_avaliacoes=100)
melhores_params, tentativas = otimizador.otimiza_hiperparams(X_aberto_continuo, y, tam_teste=.3)

0%| | 0/100 [00:00<?, ?trial/s, best loss=?] 1%| | 1/100 [00:41<1:07:55, 41.17s/trial, best loss: -0.5721084139137221] 2%|▏ | 2/100 [00:43<29:37, 18.13s/trial, best loss: -0.7127242237032704]  3%|▎ | 3/100 [00:45<17:24, 10.77s/trial, best loss: -0.7190985817778787] 4%|▍ | 4/100 [00:49<12:57, 8.10s/trial, best loss: -0.7273112668588485] 5%|▌ | 5/100 [00:52<09:55, 6.26s/trial, best loss: -0.7273112668588485] 6%|▌ | 6/100 [00:58<09:40, 6.18s/trial, best loss: -0.7273112668588485] 7%|▋ | 7/100 [01:16<15:34, 10.05s/trial, best loss: -0.7273112668588485] 8%|▊ | 8/100 [01:19<11:58, 7.81s/trial, best loss: -0.7273112668588485] 10%|█ | 10/100 [01:24<07:55, 5.28s/trial, best loss: -0.7273112668588485] 11%|█ | 11/100 [01:28<07:21, 4.96s/trial, best loss: -0.7273112668588485] 12%|█▏ | 12/100 [01:29<05:44, 3.91s/trial, best loss: -0.7273112668588485] 13%|█▎ | 13/100 [01:31<04:54, 3.39s/trial, best loss: -0.7273112668588485] 14%|█▍ | 14/100 [01:34<04:42, 3.28s/trial, best loss: -0.7273112668588485] 15%|█▌ | 15/100 [01:36<04:07, 2.91s/trial, best loss: -0.7273112668588485] 16%|█▌ | 16/100 [01:37<03:18, 2.36s/trial, best loss: -0.7273112668588485] 17%|█▋ | 17/100 [01:39<03:07, 2.26s/trial, best loss: -0.7273112668588485] 18%|█▊ | 18/100 [01:41<02:59, 2.18s/trial, best loss: -0.7273112668588485] 20%|██ | 20/100 [01:45<02:48, 2.10s/trial, best loss: -0.7273112668588485] 21%|██ | 21/100 [01:46<02:24, 1.83s/trial, best loss: -0.7273112668588485] 22%|██▏ | 22/100 [01:48<02:26, 1.88s/trial, best loss: -0.7273112668588485] 24%|██▍ | 24/100 [01:49<01:38, 1.29s/trial, best loss: -0.7273112668588485] 25%|██▌ | 25/100 [01:54<02:42, 2.16s/trial, best loss: -0.7273112668588485] 26%|██▌ | 26/100 [01:55<02:18, 1.87s/trial, best loss: -0.7273112668588485] 27%|██▋ | 27/100 [01:56<02:00, 1.65s/trial, best loss: -0.7273112668588485] 28%|██▊ | 28/100 [01:57<01:45, 1.47s/trial, best loss: -0.7273112668588485] 29%|██▉ | 29/100 [01:58<01:35, 1.34s/trial, best loss: -0.7273112668588485] 30%|███ | 30/100 [02:00<01:47, 1.53s/trial, best loss: -0.7273112668588485] 32%|███▏ | 32/100 [02:02<01:28, 1.30s/trial, best loss: -0.7276802960494829] 34%|███▍ | 34/100 [02:05<01:30, 1.38s/trial, best loss: -0.7276802960494829] 35%|███▌ | 35/100 [02:06<01:24, 1.30s/trial, best loss: -0.728035552573414]  36%|███▌ | 36/100 [02:09<01:49, 1.71s/trial, best loss: -0.728035552573414] 37%|███▋ | 37/100 [02:10<01:36, 1.53s/trial, best loss: -0.728035552573414] 38%|███▊ | 38/100 [02:13<01:59, 1.92s/trial, best loss: -0.728035552573414] 39%|███▉ | 39/100 [02:14<01:41, 1.67s/trial, best loss: -0.728035552573414] 40%|████ | 40/100 [02:15<01:29, 1.48s/trial, best loss: -0.728035552573414] 42%|████▏ | 42/100 [02:18<01:26, 1.49s/trial, best loss: -0.728035552573414] 43%|████▎ | 43/100 [02:21<01:47, 1.89s/trial, best loss: -0.728035552573414] 44%|████▍ | 44/100 [02:22<01:33, 1.67s/trial, best loss: -0.728035552573414] 45%|████▌ | 45/100 [02:26<02:06, 2.30s/trial, best loss: -0.728035552573414] 46%|████▌ | 46/100 [02:42<05:29, 6.10s/trial, best loss: -0.728035552573414] 47%|████▋ | 47/100 [02:44<04:21, 4.94s/trial, best loss: -0.728035552573414] 48%|████▊ | 48/100 [02:56<06:03, 6.98s/trial, best loss: -0.728035552573414] 49%|████▉ | 49/100 [02:58<04:42, 5.53s/trial, best loss: -0.728035552573414] 50%|█████ | 50/100 [02:59<03:29, 4.20s/trial, best loss: -0.728035552573414] 51%|█████ | 51/100 [03:05<03:52, 4.74s/trial, best loss: -0.728035552573414] 52%|█████▏ | 52/100 [03:15<05:02, 6.31s/trial, best loss: -0.728035552573414] 53%|█████▎ | 53/100 [03:18<04:10, 5.33s/trial, best loss: -0.728035552573414] 54%|█████▍ | 54/100 [03:29<05:23, 7.03s/trial, best loss: -0.728035552573414] 56%|█████▌ | 56/100 [03:31<03:07, 4.25s/trial, best loss: -0.728035552573414] 60%|██████ | 60/100 [03:32<01:19, 1.98s/trial, best loss: -0.728035552573414] 61%|██████ | 61/100 [03:33<01:10, 1.82s/trial, best loss: -0.728035552573414] 63%|██████▎ | 63/100 [03:34<00:51, 1.39s/

In [0]:
modelo_ab_continuo = otimizador.modelo
target = 'inad30'
X_train, X_test, y_train, y_test = train_test_split(X_aberto_continuo, y, train_size=.7, random_state=random_state)

temp_train=X_train.join(y_train,rsuffix='r')
temp_train[f'yhat_{prefixo}']=modelo_ab_continuo.predict_proba(X_train[aberto_continuo])[:,1]
print("KS treino: " , calc_perf(temp_train,f'yhat_{prefixo}', target))
  
temp_test=X_test.join(y_test,rsuffix='r')
temp_test[f'yhat_{prefixo}']=modelo_ab_continuo.predict_proba(temp_test[aberto_continuo])[:,1]
print("KS teste: " , calc_perf(temp_test,f'yhat_{prefixo}', target))

KS treino: (37.7596955650053, 53218, 2.3356758991318727, 51975, 1243)
KS teste: (33.510262381149936, 22809, 2.24472795826209, 22297, 512)

In [0]:
cols = aberto_continuo
modelos = [prefixo, *cols]

tab_KS = calcula_KS(modelo_ab_continuo, X_train, X_test, y_train, y_test, target, nscores=cols, continuo=aberto_continuo, prefixo=prefixo, safras=safras_dev, base_pd=base_cli_pd, base_cat=base_cli_cat)
tab_KS

vis.tabela_qs(modelos, "Safra_main", safras_dev)
vis.tabela_qs(modelos, "Safra_main", safras_nov)

col_0,ab_aberto_continuo,PP_PD_VPG01_PF_C,PP_PD_CPG01_PF_C,PP_HA_AMP01_PF,GC_AV_GRV01_PF,PF_CP_VOL04_PF_C,PF_CP_VOL04_PF,PP_AV_GRV04_PF,PP_HA_AMP02_PF,PF_CP_VOL02_PF,PF_CP_VOL01_PF,PP_PD_CPG02_PF,PF_CP_VOL03_PF,PP_AV_AMP02_PF,PP_HA_VLR02_PF,PP_AV_GRV02_PF,PF_PC_CPR02_PF,RM_PR_MOD04_PF,QtMesesEmprego,QtDiasAtrasoMinCtol1,QtDiasAtrasoMedCtol1,QtDiasAtrasoMaxCtol1,QtAtrasoAtual_Cto1,QtDiasAtrasoMaxCtol1_C,QtAtrasoMed_Cpf_C,QtAtrasoMax_Cpf_C,QtDiasAtrasoMinCtol1_C,score_ajust,P4_BVS,QuodScore_v2,SCORE_HSPN
qs_ab_aberto_continuo,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,4.600568,1.444619,1.480141,1.355819,1.398904,1.485445,2.514721,1.589307,1.348864,2.681355,3.179992,2.615804,2.500866,1.487710,1.579494,1.601888,2.479224,2.462022,1.824315,1.775703,1.746448,1.746448,1.858934,1.454042,1.478783,1.475401,1.858934,2.864387,4.560485,4.021724,1.250390
1,3.547426,2.776336,2.584345,1.583732,1.536948,2.401693,2.202979,1.115919,1.396917,2.292720,2.498789,2.111260,1.974778,2.020662,1.787694,2.450832,2.051282,2.000238,2.082601,4.349689,1.181369,1.227886,NaN,3.308584,2.447946,2.074496,NaN,2.477132,2.985075,3.066038,3.610406
2,2.625563,4.007833,3.436999,2.123756,1.614298,4.003246,1.891630,1.511668,1.530222,1.932133,2.086366,1.994747,1.884309,2.329003,1.849799,2.484255,1.928527,0.716746,1.945869,NaN,1.176648,1.190420,NaN,5.789909,4.305527,2.497725,NaN,2.064170,2.387195,2.391350,2.975697
3,1.829395,NaN,4.003246,1.816076,2.127955,NaN,1.661590,1.925581,1.574493,1.947872,2.111750,1.696353,1.737144,2.214533,1.856984,2.019151,1.986259,2.794630,2.062374,NaN,1.251539,1.067476,NaN,NaN,NaN,3.909360,NaN,1.930978,1.965824,2.005402,2.598036
4,1.593570,NaN,NaN,2.043011,2.236842,NaN,1.611676,2.540835,1.969365,1.637306,1.865723,1.507974,1.408158,3.441266,2.100608,1.573439,1.778937,3.614458,1.848136,NaN,1.260716,1.746373,NaN,NaN,NaN,NaN,NaN,2.098246,1.606453,1.687909,1.943281
5,1.316428,NaN,NaN,2.477748,2.309501,NaN,1.288907,3.230073,2.291254,1.636526,1.517566,1.635530,1.471293,NaN,2.710963,2.716186,1.476533,NaN,1.912124,NaN,1.754017,1.681311,NaN,NaN,NaN,NaN,NaN,1.641301,1.281348,1.505525,1.670426
6,1.247142,NaN,NaN,3.306316,3.242867,NaN,1.321295,NaN,2.519609,1.341699,1.427186,1.372578,1.341299,NaN,1.661637,NaN,3.653342,NaN,1.637066,NaN,2.067967,1.933066,NaN,NaN,NaN,NaN,NaN,1.585863,1.362398,1.270884,1.452768
7,0.935356,NaN,NaN,NaN,NaN,NaN,2.768575,NaN,2.012206,1.371410,0.969261,1.379167,2.722868,NaN,3.614458,NaN,NaN,NaN,1.617474,NaN,2.563567,2.694743,NaN,NaN,NaN,NaN,NaN,1.378556,0.976482,0.927411,1.105756
8,0.595857,NaN,NaN,NaN,NaN,NaN,3.614458,NaN,3.653342,3.677173,0.652943,1.465841,3.614458,NaN,NaN,NaN,NaN,NaN,1.685147,NaN,3.624896,3.593739,NaN,NaN,NaN,NaN,NaN,1.301170,0.779823,1.009891,0.694591


### XgBoost

#### Restrito

In [0]:
prefixo = "xg_restrito"
prefixos.append(prefixo)

otimizador = OtimizadorBayesiano(classificador="xgboost", n_estimadores = 100, random_state=random_state, max_avaliacoes=100)
melhores_params, tentativas = otimizador.otimiza_hiperparams(X_restrito, y, tam_teste=.3)

0%| | 0/100 [00:00<?, ?trial/s, best loss=?] 1%| | 1/100 [00:50<1:23:46, 50.78s/trial, best loss: -0.6635809417326322] 2%|▏ | 2/100 [00:55<38:58, 23.86s/trial, best loss: -0.6844720737374983]  4%|▍ | 4/100 [00:59<16:15, 10.16s/trial, best loss: -0.6844720737374983] 5%|▌ | 5/100 [01:04<13:34, 8.57s/trial, best loss: -0.6844720737374983] 6%|▌ | 6/100 [01:07<10:45, 6.87s/trial, best loss: -0.6844720737374983] 7%|▋ | 7/100 [01:13<10:14, 6.61s/trial, best loss: -0.6844720737374983] 9%|▉ | 9/100 [01:16<06:25, 4.24s/trial, best loss: -0.6844720737374983] 10%|█ | 10/100 [01:27<08:53, 5.93s/trial, best loss: -0.6844720737374983] 11%|█ | 11/100 [01:28<06:52, 4.64s/trial, best loss: -0.6844720737374983] 12%|█▏ | 12/100 [01:30<05:45, 3.93s/trial, best loss: -0.6844720737374983] 13%|█▎ | 13/100 [01:37<06:57, 4.79s/trial, best loss: -0.6844720737374983] 14%|█▍ | 14/100 [01:42<06:57, 4.86s/trial, best loss: -0.6844720737374983] 15%|█▌ | 15/100 [01:54<09:25, 6.65s/trial, best loss: -0.6844720737374983] 16%|█▌ | 16/100 [01:56<07:24, 5.29s/trial, best loss: -0.6844720737374983] 17%|█▋ | 17/100 [01:59<06:23, 4.62s/trial, best loss: -0.6844720737374983] 18%|█▊ | 18/100 [02:01<05:15, 3.85s/trial, best loss: -0.6844720737374983] 19%|█▉ | 19/100 [02:10<07:16, 5.39s/trial, best loss: -0.6844720737374983] 20%|██ | 20/100 [02:12<05:50, 4.39s/trial, best loss: -0.6865498914512042] 21%|██ | 21/100 [02:13<04:26, 3.38s/trial, best loss: -0.6865498914512042] 22%|██▏ | 22/100 [02:16<04:15, 3.27s/trial, best loss: -0.6867933641577342] 23%|██▎ | 23/100 [02:18<03:43, 2.90s/trial, best loss: -0.6867933641577342] 24%|██▍ | 24/100 [02:32<07:53, 6.24s/trial, best loss: -0.6867933641577342] 25%|██▌ | 25/100 [02:36<06:57, 5.57s/trial, best loss: -0.6867933641577342] 27%|██▋ | 27/100 [02:42<05:20, 4.39s/trial, best loss: -0.6867933641577342] 28%|██▊ | 28/100 [02:44<04:33, 3.81s/trial, best loss: -0.6867933641577342] 30%|███ | 30/100 [02:45<02:50, 2.44s/trial, best loss: -0.6867933641577342] 31%|███ | 31/100 [02:46<02:25, 2.11s/trial, best loss: -0.6867933641577342] 32%|███▏ | 32/100 [02:48<02:22, 2.09s/trial, best loss: -0.6867933641577342] 33%|███▎ | 33/100 [02:49<02:01, 1.81s/trial, best loss: -0.6867933641577342] 34%|███▍ | 34/100 [02:52<02:21, 2.14s/trial, best loss: -0.6886080876911692] 35%|███▌ | 35/100 [02:58<03:29, 3.22s/trial, best loss: -0.6886080876911692] 36%|███▌ | 36/100 [03:03<03:59, 3.74s/trial, best loss: -0.6886080876911692] 37%|███▋ | 37/100 [03:07<04:00, 3.82s/trial, best loss: -0.6886080876911692] 38%|███▊ | 38/100 [03:08<03:05, 3.00s/trial, best loss: -0.6886080876911692] 39%|███▉ | 39/100 [03:09<02:27, 2.41s/trial, best loss: -0.6886080876911692] 40%|████ | 40/100 [03:14<03:11, 3.19s/trial, best loss: -0.6886080876911692] 41%|████ | 41/100 [03:15<02:29, 2.54s/trial, best loss: -0.6886080876911692] 42%|████▏ | 42/100 [03:17<02:18, 2.39s/trial, best loss: -0.6886080876911692] 44%|████▍ | 44/100 [03:26<03:08, 3.36s/trial, best loss: -0.6886080876911692] 45%|████▌ | 45/100 [03:31<03:27, 3.78s/trial, best loss: -0.6886080876911692] 46%|████▌ | 46/100 [03:33<02:59, 3.32s/trial, best loss: -0.6886080876911692] 47%|████▋ | 47/100 [03:34<02:22, 2.69s/trial, best loss: -0.6886080876911692] 48%|████▊ | 48/100 [03:35<01:55, 2.22s/trial, best loss: -0.6886080876911692] 49%|████▉ | 49/100 [03:41<02:48, 3.31s/trial, best loss: -0.6886080876911692] 50%|█████ | 50/100 [03:42<02:12, 2.64s/trial, best loss: -0.6886080876911692] 51%|█████ | 51/100 [03:47<02:43, 3.34s/trial, best loss: -0.6886080876911692] 52%|█████▏ | 52/100 [03:48<02:07, 2.66s/trial, best loss: -0.6886080876911692] 53%|█████▎ | 53/100 [03:49<01:42, 2.17s/trial, best loss: -0.6886080876911692] 54%|█████▍ | 54/100 [03:52<01:51, 2.43s/trial, best loss: -0.6886080876911692] 55%|█████▌ | 55/100 [03:54<01:43, 2.30s/trial, best loss: -0.6886080876911692] 56%|█████▌ | 56/100 [03:59<02:17, 3.12s/trial, best loss: -0.6886080876911692] 57%|█████▋ | 57/100 [

In [0]:
modelo_xg = otimizador.modelo
target = 'inad30'
X_train, X_test, y_train, y_test = train_test_split(X_restrito, y, train_size=.7, random_state=random_state)

temp_train=X_train.join(y_train,rsuffix='r')
temp_train[f'yhat_{prefixo}']=modelo_xg.predict_proba(X_train[restrito])[:,1]
print("KS treino: " , calc_perf(temp_train,f'yhat_{prefixo}', target))
  
temp_test=X_test.join(y_test,rsuffix='r')
temp_test[f'yhat_{prefixo}']=modelo_xg.predict_proba(temp_test[restrito])[:,1]
print("KS teste: " , calc_perf(temp_test,f'yhat_{prefixo}', target))

KS treino: (33.5183950759172, 53218, 2.3356758991318727, 51975, 1243)
KS teste: (26.651690109656013, 22809, 2.24472795826209, 22297, 512)

In [0]:
cols = restrito
modelos = [prefixo, *cols]

tab_KS = calcula_KS(modelo_xg, X_train, X_test, y_train, y_test, target, nscores=cols, restrito=restrito, prefixo=prefixo, safras=safras_dev, base_pd=base_cli_pd, base_cat=base_cli_cat)
tab_KS

vis.tabela_qs(modelos, "Safra_main", safras_dev)
vis.tabela_qs(modelos, "Safra_main", safras_nov)

col_0,xg_restrito,WoE_score_ajust,WoE_PF_CP_VOL04_PF_C,P4_BVS,QuodScore_v2,SCORE_HSPN
qs_xg_restrito,,,,,,
0,4.482783,2.467365,2.514751,4.560485,4.021724,1.250390
1,2.993141,1.924554,1.718287,2.985075,3.066038,3.610406
2,2.334927,1.584174,1.327765,2.387195,2.391350,2.975697
3,1.773713,1.427862,NaN,1.965824,2.005402,2.598036
4,1.586642,1.252926,NaN,1.606453,1.687909,1.943281
5,1.364928,NaN,NaN,1.281348,1.505525,1.670426
6,1.551676,NaN,NaN,1.362398,1.270884,1.452768
7,1.261261,NaN,NaN,0.976482,0.927411,1.105756
8,0.865891,NaN,NaN,0.779823,1.009891,0.694591


#### Aberto

In [0]:
prefixo = "xg_aberto"
prefixos.append(prefixo)

otimizador = OtimizadorBayesiano(classificador="xgboost", n_estimadores = 100, random_state=random_state, max_avaliacoes=100)
melhores_params, tentativas = otimizador.otimiza_hiperparams(X_aberto, y, tam_teste=.3)

0%| | 0/100 [00:00<?, ?trial/s, best loss=?] 1%| | 1/100 [00:45<1:14:49, 45.35s/trial, best loss: -0.7125397159651523] 2%|▏ | 2/100 [00:53<38:12, 23.39s/trial, best loss: -0.7125397159651523]  3%|▎ | 3/100 [01:00<25:43, 15.92s/trial, best loss: -0.7125397159651523] 4%|▍ | 4/100 [01:04<17:56, 11.22s/trial, best loss: -0.7125397159651523] 5%|▌ | 5/100 [01:05<11:56, 7.54s/trial, best loss: -0.7125397159651523] 6%|▌ | 6/100 [01:08<09:24, 6.00s/trial, best loss: -0.7125397159651523] 7%|▋ | 7/100 [01:09<06:46, 4.37s/trial, best loss: -0.7125397159651523] 8%|▊ | 8/100 [01:42<20:41, 13.50s/trial, best loss: -0.7125397159651523] 9%|▉ | 9/100 [01:43<14:33, 9.60s/trial, best loss: -0.7136741700116608] 11%|█ | 11/100 [01:46<08:34, 5.78s/trial, best loss: -0.7136741700116608] 12%|█▏ | 12/100 [01:48<07:05, 4.84s/trial, best loss: -0.7136741700116608] 13%|█▎ | 13/100 [01:53<07:04, 4.88s/trial, best loss: -0.7136741700116608] 14%|█▍ | 14/100 [01:55<05:52, 4.10s/trial, best loss: -0.7136741700116608] 15%|█▌ | 15/100 [01:57<04:58, 3.51s/trial, best loss: -0.7136741700116608] 16%|█▌ | 16/100 [01:58<03:54, 2.79s/trial, best loss: -0.7136741700116608] 18%|█▊ | 18/100 [02:01<03:01, 2.22s/trial, best loss: -0.7136741700116608] 19%|█▉ | 19/100 [02:05<03:35, 2.66s/trial, best loss: -0.7136741700116608] 20%|██ | 20/100 [02:06<02:58, 2.23s/trial, best loss: -0.7136741700116608] 21%|██ | 21/100 [02:11<03:55, 2.98s/trial, best loss: -0.7136741700116608] 22%|██▏ | 22/100 [02:13<03:31, 2.72s/trial, best loss: -0.7164771500930619] 23%|██▎ | 23/100 [02:16<03:35, 2.80s/trial, best loss: -0.7164771500930619] 24%|██▍ | 24/100 [02:22<04:43, 3.73s/trial, best loss: -0.7164771500930619] 25%|██▌ | 25/100 [02:23<03:40, 2.93s/trial, best loss: -0.7164771500930619] 26%|██▌ | 26/100 [02:33<06:11, 5.03s/trial, best loss: -0.7164771500930619] 27%|██▋ | 27/100 [02:46<08:59, 7.39s/trial, best loss: -0.7164771500930619] 28%|██▊ | 28/100 [02:48<06:57, 5.80s/trial, best loss: -0.7164771500930619] 29%|██▉ | 29/100 [02:58<08:21, 7.06s/trial, best loss: -0.7164771500930619] 30%|███ | 30/100 [03:02<07:10, 6.15s/trial, best loss: -0.7164771500930619] 31%|███ | 31/100 [03:03<05:18, 4.61s/trial, best loss: -0.7164771500930619] 32%|███▏ | 32/100 [03:04<04:00, 3.53s/trial, best loss: -0.7164771500930619] 33%|███▎ | 33/100 [03:05<03:06, 2.78s/trial, best loss: -0.7164771500930619] 34%|███▍ | 34/100 [03:09<03:28, 3.16s/trial, best loss: -0.7164771500930619] 35%|███▌ | 35/100 [03:12<03:22, 3.12s/trial, best loss: -0.7164771500930619] 36%|███▌ | 36/100 [03:16<03:17, 3.09s/trial, best loss: -0.7164771500930619] 37%|███▋ | 37/100 [03:19<03:13, 3.07s/trial, best loss: -0.7164771500930619] 38%|███▊ | 38/100 [03:20<02:31, 2.45s/trial, best loss: -0.7164771500930619] 39%|███▉ | 39/100 [03:24<02:58, 2.92s/trial, best loss: -0.7164771500930619] 40%|████ | 40/100 [03:26<02:39, 2.65s/trial, best loss: -0.7164771500930619] 41%|████ | 41/100 [03:31<03:18, 3.36s/trial, best loss: -0.7164771500930619] 42%|████▏ | 42/100 [03:37<04:01, 4.16s/trial, best loss: -0.7164771500930619] 43%|████▎ | 43/100 [03:39<03:20, 3.52s/trial, best loss: -0.7164771500930619] 44%|████▍ | 44/100 [03:46<04:15, 4.57s/trial, best loss: -0.7164771500930619] 45%|████▌ | 45/100 [03:50<04:02, 4.41s/trial, best loss: -0.7164771500930619] 47%|████▋ | 47/100 [03:59<03:56, 4.45s/trial, best loss: -0.7164771500930619] 48%|████▊ | 48/100 [04:09<05:03, 5.84s/trial, best loss: -0.7164771500930619] 49%|████▉ | 49/100 [04:12<04:20, 5.11s/trial, best loss: -0.7164771500930619] 51%|█████ | 51/100 [04:14<02:44, 3.36s/trial, best loss: -0.7164771500930619] 52%|█████▏ | 52/100 [04:17<02:37, 3.28s/trial, best loss: -0.7164771500930619] 53%|█████▎ | 53/100 [04:19<02:19, 2.97s/trial, best loss: -0.7164771500930619] 54%|█████▍ | 54/100 [04:34<04:42, 6.15s/trial, best loss: -0.7164771500930619] 55%|█████▌ | 55/100 [04:35<03:33, 4.74s/trial, best loss: -0.7164771500930619] 57%|█████▋ | 57/100 [04:36<02

In [0]:
modelo_xg_aberto = otimizador.modelo
target = 'inad30'
X_train, X_test, y_train, y_test = train_test_split(X_aberto, y, train_size=.7, random_state=random_state)

temp_train=X_train.join(y_train,rsuffix='r')
temp_train[f'yhat_{prefixo}']=modelo_xg_aberto.predict_proba(X_train[aberto])[:,1]
print("KS treino: " , calc_perf(temp_train,f'yhat_{prefixo}', target))
  
temp_test=X_test.join(y_test,rsuffix='r')
temp_test[f'yhat_{prefixo}']=modelo_xg_aberto.predict_proba(temp_test[aberto])[:,1]
print("KS teste: " , calc_perf(temp_test,f'yhat_{prefixo}', target))

KS treino: (45.894869470090704, 53218, 2.3356758991318727, 51975, 1243)
KS teste: (31.389610289500823, 22809, 2.24472795826209, 22297, 512)

In [0]:
cols = aberto
modelos = [prefixo, *cols]

tab_KS = calcula_KS(modelo_xg_aberto, X_train, X_test, y_train, y_test, target, nscores=cols, aberto=aberto, prefixo=prefixo, safras=safras_dev, base_pd=base_cli_pd, base_cat=base_cli_cat)
tab_KS

vis.tabela_qs(modelos, "Safra_main", safras_dev)
vis.tabela_qs(modelos, "Safra_main", safras_nov)

col_0,xg_aberto,WoE_PF_CP_VOL04_PF_C,WoE_PF_CP_VOL02_PF_C,WoE_PF_CP_VOL01_PF_C,WoE_PF_CP_VOL03_PF_C,WoE_PF_PC_CPR02_PF_C,WoE_QtDiasAtrasoMaxCtol1_C,WoE_QtAtrasoMed_Cpf_C,WoE_QtAtrasoMax_Cpf_C,WoE_QtDiasAtrasoMinCtol1_C,WoE_score_ajust,P4_BVS,QuodScore_v2,SCORE_HSPN
qs_xg_aberto,,,,,,,,,,,,,,
0,4.039354,2.514751,2.834066,3.240335,2.561404,2.826150,2.549270,2.919095,2.932939,1.858934,2.467365,4.560485,4.021724,1.250390
1,3.457355,1.718287,1.969660,2.254973,1.903100,1.930585,1.214792,1.950833,2.074496,NaN,1.924554,2.985075,3.066038,3.610406
2,2.792212,1.327765,1.432791,1.269113,1.388214,1.476533,NaN,1.359164,1.547718,NaN,1.584174,2.387195,2.391350,2.975697
3,2.237927,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.407948,NaN,1.427862,1.965824,2.005402,2.598036
4,1.732142,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.252926,1.606453,1.687909,1.943281
5,1.448070,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.281348,1.505525,1.670426
6,1.198642,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.362398,1.270884,1.452768
7,0.845285,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.976482,0.927411,1.105756
8,0.609714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.779823,1.009891,0.694591


#### Aberto Contínuo

In [0]:
prefixo = "xg_aberto_continuo"
prefixos.append(prefixo)

otimizador = OtimizadorBayesiano(classificador="xgboost", n_estimadores = 100, random_state=random_state, max_avaliacoes=100)
melhores_params, tentativas = otimizador.otimiza_hiperparams(X_aberto_continuo, y, tam_teste=.3)

0%| | 0/100 [00:00<?, ?trial/s, best loss=?] 1%| | 1/100 [00:44<1:13:01, 44.26s/trial, best loss: -0.7165753012596986] 2%|▏ | 2/100 [00:50<35:32, 21.76s/trial, best loss: -0.7165753012596986]  3%|▎ | 3/100 [01:10<33:54, 20.97s/trial, best loss: -0.7165753012596986] 4%|▍ | 4/100 [01:11<20:56, 13.09s/trial, best loss: -0.7165753012596986] 5%|▌ | 5/100 [01:52<36:41, 23.18s/trial, best loss: -0.7165753012596986] 6%|▌ | 6/100 [01:54<25:02, 15.98s/trial, best loss: -0.7165753012596986] 7%|▋ | 7/100 [01:55<17:11, 11.09s/trial, best loss: -0.7165753012596986] 8%|▊ | 8/100 [01:59<13:33, 8.84s/trial, best loss: -0.7165753012596986] 9%|▉ | 9/100 [02:17<17:45, 11.71s/trial, best loss: -0.7165753012596986] 10%|█ | 10/100 [02:24<15:23, 10.26s/trial, best loss: -0.7165753012596986] 11%|█ | 11/100 [02:29<12:50, 8.66s/trial, best loss: -0.7165753012596986] 12%|█▏ | 12/100 [02:35<11:31, 7.86s/trial, best loss: -0.7165753012596986] 13%|█▎ | 13/100 [02:43<11:27, 7.91s/trial, best loss: -0.7165753012596986] 14%|█▍ | 14/100 [02:55<13:06, 9.15s/trial, best loss: -0.7165753012596986] 15%|█▌ | 15/100 [02:59<10:46, 7.60s/trial, best loss: -0.7165753012596986] 16%|█▌ | 16/100 [03:01<08:17, 5.92s/trial, best loss: -0.7165753012596986] 17%|█▋ | 17/100 [03:15<11:33, 8.36s/trial, best loss: -0.7172176417371171] 18%|█▊ | 18/100 [03:22<10:52, 7.96s/trial, best loss: -0.7172176417371171] 19%|█▉ | 19/100 [03:28<09:57, 7.38s/trial, best loss: -0.7172176417371171] 20%|██ | 20/100 [03:29<07:17, 5.47s/trial, best loss: -0.7172176417371171] 21%|██ | 21/100 [03:30<05:25, 4.13s/trial, best loss: -0.7172176417371171] 22%|██▏ | 22/100 [03:39<07:16, 5.60s/trial, best loss: -0.7172176417371171] 23%|██▎ | 23/100 [03:53<10:25, 8.13s/trial, best loss: -0.7172176417371171] 24%|██▍ | 24/100 [04:18<16:43, 13.20s/trial, best loss: -0.7172176417371171] 25%|██▌ | 25/100 [04:27<14:58, 11.98s/trial, best loss: -0.7172176417371171] 26%|██▌ | 26/100 [04:28<10:42, 8.69s/trial, best loss: -0.7172176417371171] 27%|██▋ | 27/100 [04:29<07:46, 6.39s/trial, best loss: -0.7172176417371171] 28%|██▊ | 28/100 [04:43<10:25, 8.68s/trial, best loss: -0.7172176417371171] 29%|██▉ | 29/100 [05:02<13:35, 11.49s/trial, best loss: -0.7172176417371171] 30%|███ | 30/100 [05:08<11:29, 9.85s/trial, best loss: -0.7172176417371171] 31%|███ | 31/100 [05:09<08:16, 7.19s/trial, best loss: -0.7172176417371171] 33%|███▎ | 33/100 [05:10<04:35, 4.11s/trial, best loss: -0.7172176417371171] 34%|███▍ | 34/100 [05:24<07:13, 6.58s/trial, best loss: -0.7172176417371171] 35%|███▌ | 35/100 [05:25<05:32, 5.12s/trial, best loss: -0.7172176417371171] 36%|███▌ | 36/100 [05:27<04:33, 4.28s/trial, best loss: -0.7172176417371171] 38%|███▊ | 38/100 [05:31<03:23, 3.29s/trial, best loss: -0.7172176417371171] 39%|███▉ | 39/100 [05:35<03:31, 3.46s/trial, best loss: -0.7172176417371171] 40%|████ | 40/100 [05:42<04:22, 4.37s/trial, best loss: -0.7172176417371171] 41%|████ | 41/100 [05:47<04:27, 4.54s/trial, best loss: -0.7172176417371171] 42%|████▏ | 42/100 [05:56<05:35, 5.78s/trial, best loss: -0.7172176417371171] 43%|████▎ | 43/100 [06:03<05:49, 6.13s/trial, best loss: -0.7172176417371171] 44%|████▍ | 44/100 [06:17<07:50, 8.40s/trial, best loss: -0.7172176417371171] 45%|████▌ | 45/100 [06:21<06:34, 7.17s/trial, best loss: -0.7172176417371171] 46%|████▌ | 46/100 [06:38<09:03, 10.07s/trial, best loss: -0.7172176417371171] 47%|████▋ | 47/100 [06:53<10:11, 11.54s/trial, best loss: -0.7172176417371171] 49%|████▉ | 49/100 [07:01<06:52, 8.09s/trial, best loss: -0.7175710910520473] 51%|█████ | 51/100 [07:02<04:09, 5.09s/trial, best loss: -0.7175710910520473] 52%|█████▏ | 52/100 [07:30<08:07, 10.16s/trial, best loss: -0.7175710910520473] 53%|█████▎ | 53/100 [07:39<07:44, 9.89s/trial, best loss: -0.7175710910520473] 54%|█████▍ | 54/100 [07:41<06:02, 7.88s/trial, best loss: -0.7175710910520473] 55%|█████▌ | 55/100 [07:50<06:08, 8.19s/trial, best loss: -0.7175710910520473] 56%|█████▌ | 56/100 [07:

In [0]:
modelo_xg_continuo = otimizador.modelo
target = 'inad30'
X_train, X_test, y_train, y_test = train_test_split(X_aberto_continuo, y, train_size=.7, random_state=random_state)

temp_train=X_train.join(y_train,rsuffix='r')
temp_train[f'yhat_{prefixo}']=modelo_xg_continuo.predict_proba(X_train[aberto_continuo])[:,1]
print("KS treino: " , calc_perf(temp_train,f'yhat_{prefixo}', target))
  
temp_test=X_test.join(y_test,rsuffix='r')
temp_test[f'yhat_{prefixo}']=modelo_xg_continuo.predict_proba(temp_test[aberto_continuo])[:,1]
print("KS teste: " , calc_perf(temp_test,f'yhat_{prefixo}', target))

KS treino: (54.01822693858092, 53218, 2.3356758991318727, 51975, 1243)
KS teste: (33.25430726386958, 22809, 2.24472795826209, 22297, 512)

In [0]:
cols = aberto_continuo
modelos = [prefixo, *cols]

tab_KS = calcula_KS(modelo_xg_continuo, X_train, X_test, y_train, y_test, target, nscores=cols, continuo=aberto_continuo, prefixo=prefixo, safras=safras_dev, base_pd=base_cli_pd, base_cat=base_cli_cat)
tab_KS

vis.tabela_qs(modelos, "Safra_main", safras_dev)
vis.tabela_qs(modelos, "Safra_main", safras_nov)

col_0,xg_aberto_continuo,PP_PD_VPG01_PF_C,PP_PD_CPG01_PF_C,PP_HA_AMP01_PF,GC_AV_GRV01_PF,PF_CP_VOL04_PF_C,PF_CP_VOL04_PF,PP_AV_GRV04_PF,PP_HA_AMP02_PF,PF_CP_VOL02_PF,PF_CP_VOL01_PF,PP_PD_CPG02_PF,PF_CP_VOL03_PF,PP_AV_AMP02_PF,PP_HA_VLR02_PF,PP_AV_GRV02_PF,PF_PC_CPR02_PF,RM_PR_MOD04_PF,QtMesesEmprego,QtDiasAtrasoMinCtol1,QtDiasAtrasoMedCtol1,QtDiasAtrasoMaxCtol1,QtAtrasoAtual_Cto1,QtDiasAtrasoMaxCtol1_C,QtAtrasoMed_Cpf_C,QtAtrasoMax_Cpf_C,QtDiasAtrasoMinCtol1_C,score_ajust,P4_BVS,QuodScore_v2,SCORE_HSPN
qs_xg_aberto_continuo,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,6.478210,1.444619,1.480141,1.355819,1.398904,1.485445,2.514721,1.589307,1.348864,2.681355,3.179992,2.615804,2.500866,1.487710,1.579494,1.601888,2.479224,2.462022,1.824315,1.775703,1.746448,1.746448,1.858934,1.454042,1.478783,1.475401,1.858934,2.864387,4.560485,4.021724,1.250390
1,3.083212,2.776336,2.584345,1.583732,1.536948,2.401693,2.202979,1.115919,1.396917,2.292720,2.498789,2.111260,1.974778,2.020662,1.787694,2.450832,2.051282,2.000238,2.082601,4.349689,1.181369,1.227886,NaN,3.308584,2.447946,2.074496,NaN,2.477132,2.985075,3.066038,3.610406
2,2.251784,4.007833,3.436999,2.123756,1.614298,4.003246,1.891630,1.511668,1.530222,1.932133,2.086366,1.994747,1.884309,2.329003,1.849799,2.484255,1.928527,0.716746,1.945869,NaN,1.176648,1.190420,NaN,5.789909,4.305527,2.497725,NaN,2.064170,2.387195,2.391350,2.975697
3,1.711356,NaN,4.003246,1.816076,2.127955,NaN,1.661590,1.925581,1.574493,1.947872,2.111750,1.696353,1.737144,2.214533,1.856984,2.019151,1.986259,2.794630,2.062374,NaN,1.251539,1.067476,NaN,NaN,NaN,3.909360,NaN,1.930978,1.965824,2.005402,2.598036
4,1.434213,NaN,NaN,2.043011,2.236842,NaN,1.611676,2.540835,1.969365,1.637306,1.865723,1.507974,1.408158,3.441266,2.100608,1.573439,1.778937,3.614458,1.848136,NaN,1.260716,1.746373,NaN,NaN,NaN,NaN,NaN,2.098246,1.606453,1.687909,1.943281
5,1.122428,NaN,NaN,2.477748,2.309501,NaN,1.288907,3.230073,2.291254,1.636526,1.517566,1.635530,1.471293,NaN,2.710963,2.716186,1.476533,NaN,1.912124,NaN,1.754017,1.681311,NaN,NaN,NaN,NaN,NaN,1.641301,1.281348,1.505525,1.670426
6,1.087785,NaN,NaN,3.306316,3.242867,NaN,1.321295,NaN,2.519609,1.341699,1.427186,1.372578,1.341299,NaN,1.661637,NaN,3.653342,NaN,1.637066,NaN,2.067967,1.933066,NaN,NaN,NaN,NaN,NaN,1.585863,1.362398,1.270884,1.452768
7,0.706714,NaN,NaN,NaN,NaN,NaN,2.768575,NaN,2.012206,1.371410,0.969261,1.379167,2.722868,NaN,3.614458,NaN,NaN,NaN,1.617474,NaN,2.563567,2.694743,NaN,NaN,NaN,NaN,NaN,1.378556,0.976482,0.927411,1.105756
8,0.457285,NaN,NaN,NaN,NaN,NaN,3.614458,NaN,3.653342,3.677173,0.652943,1.465841,3.614458,NaN,NaN,NaN,NaN,NaN,1.685147,NaN,3.624896,3.593739,NaN,NaN,NaN,NaN,NaN,1.301170,0.779823,1.009891,0.694591


### LightGBM

#### Restrito

In [0]:
prefixo = "lgbm_restrito"
prefixos.append(prefixo)

otimizador = OtimizadorBayesiano(classificador="light_gbm", n_estimadores = 100, random_state=random_state, max_avaliacoes=100)
melhores_params, tentativas = otimizador.otimiza_hiperparams(X_restrito, y, tam_teste=.3)

0%| | 0/100 [00:00<?, ?trial/s, best loss=?] 1%| | 1/100 [01:36<2:38:59, 96.36s/trial, best loss: -0.6767305263880791] 2%|▏ | 2/100 [02:16<1:43:17, 63.24s/trial, best loss: -0.6767305263880791] 3%|▎ | 3/100 [02:17<56:18, 34.83s/trial, best loss: -0.6767305263880791]  4%|▍ | 4/100 [02:33<43:51, 27.41s/trial, best loss: -0.6787939345820065] 5%|▌ | 5/100 [02:48<36:19, 22.94s/trial, best loss: -0.6787939345820065] 6%|▌ | 6/100 [02:50<24:47, 15.83s/trial, best loss: -0.6787939345820065] 7%|▋ | 7/100 [02:56<19:33, 12.62s/trial, best loss: -0.6787939345820065] 8%|▊ | 8/100 [03:59<43:59, 28.69s/trial, best loss: -0.6795141915812666] 9%|▉ | 9/100 [04:11<35:36, 23.48s/trial, best loss: -0.6797652413301116] 10%|█ | 10/100 [04:56<45:12, 30.14s/trial, best loss: -0.6797652413301116] 11%|█ | 11/100 [04:58<31:56, 21.53s/trial, best loss: -0.6797652413301116] 12%|█▏ | 12/100 [05:30<36:16, 24.73s/trial, best loss: -0.6797652413301116] 13%|█▎ | 13/100 [05:49<33:21, 23.01s/trial, best loss: -0.6797652413301116] 15%|█▌ | 15/100 [05:58<20:26, 14.43s/trial, best loss: -0.6797652413301116] 16%|█▌ | 16/100 [06:07<18:19, 13.09s/trial, best loss: -0.6797652413301116] 17%|█▋ | 17/100 [06:14<15:54, 11.50s/trial, best loss: -0.6797652413301116] 18%|█▊ | 18/100 [06:17<12:33, 9.19s/trial, best loss: -0.6797652413301116] 19%|█▉ | 19/100 [06:23<11:12, 8.30s/trial, best loss: -0.6797652413301116] 20%|██ | 20/100 [06:45<16:18, 12.23s/trial, best loss: -0.6797652413301116] 21%|██ | 21/100 [06:52<13:43, 10.43s/trial, best loss: -0.6797652413301116] 22%|██▏ | 22/100 [06:55<10:44, 8.26s/trial, best loss: -0.6797652413301116] 23%|██▎ | 23/100 [07:03<10:30, 8.19s/trial, best loss: -0.6797652413301116] 24%|██▍ | 24/100 [07:33<18:35, 14.67s/trial, best loss: -0.6797652413301116] 25%|██▌ | 25/100 [07:46<17:43, 14.18s/trial, best loss: -0.6797652413301116] 26%|██▌ | 26/100 [07:53<14:51, 12.05s/trial, best loss: -0.6797652413301116] 27%|██▋ | 27/100 [07:56<11:22, 9.35s/trial, best loss: -0.6797652413301116] 28%|██▊ | 28/100 [08:07<11:49, 9.85s/trial, best loss: -0.6797652413301116] 29%|██▉ | 29/100 [08:39<19:31, 16.50s/trial, best loss: -0.6797652413301116] 30%|███ | 30/100 [08:47<16:17, 13.96s/trial, best loss: -0.6797652413301116] 31%|███ | 31/100 [08:55<14:00, 12.18s/trial, best loss: -0.6797652413301116] 32%|███▏ | 32/100 [09:14<16:08, 14.24s/trial, best loss: -0.6797652413301116] 33%|███▎ | 33/100 [09:15<11:28, 10.27s/trial, best loss: -0.6797652413301116] 34%|███▍ | 34/100 [09:42<16:50, 15.31s/trial, best loss: -0.6827310183264564] 35%|███▌ | 35/100 [10:32<27:52, 25.74s/trial, best loss: -0.6827310183264564] 36%|███▌ | 36/100 [10:41<22:06, 20.72s/trial, best loss: -0.6827310183264564] 37%|███▋ | 37/100 [10:48<17:26, 16.61s/trial, best loss: -0.6827310183264564] 38%|███▊ | 38/100 [11:14<20:05, 19.44s/trial, best loss: -0.6827310183264564] 39%|███▉ | 39/100 [11:26<17:30, 17.22s/trial, best loss: -0.6827310183264564] 40%|████ | 40/100 [12:44<35:29, 35.49s/trial, best loss: -0.6827310183264564] 41%|████ | 41/100 [13:21<35:21, 35.96s/trial, best loss: -0.6827310183264564] 42%|████▏ | 42/100 [13:24<25:12, 26.08s/trial, best loss: -0.6827310183264564] 43%|████▎ | 43/100 [13:25<17:38, 18.56s/trial, best loss: -0.6827310183264564] 44%|████▍ | 44/100 [13:42<16:53, 18.11s/trial, best loss: -0.6827310183264564] 45%|████▌ | 45/100 [13:57<15:45, 17.19s/trial, best loss: -0.6827310183264564] 47%|████▋ | 47/100 [14:08<10:25, 11.80s/trial, best loss: -0.6827310183264564] 48%|████▊ | 48/100 [15:03<19:18, 22.29s/trial, best loss: -0.6827310183264564] 49%|████▉ | 49/100 [15:44<23:07, 27.20s/trial, best loss: -0.6827310183264564]

In [0]:
modelo_lgbm = otimizador.modelo
target = 'inad30'

X_train, X_test, y_train, y_test = train_test_split(X_restrito, y, train_size=.7, random_state=random_state)
 
temp_train=X_train.join(y_train,rsuffix='r')
temp_train[f'yhat_{prefixo}']=modelo_lgbm.predict_proba(X_train[restrito])[:,1]
print("KS treino: " , calc_perf(temp_train,f'yhat_{prefixo}', target))
  
temp_test=X_test.join(y_test,rsuffix='r')
temp_test[f'yhat_{prefixo}']=modelo_lgbm.predict_proba(temp_test[restrito])[:,1]
print("KS teste: " , calc_perf(temp_test,f'yhat_{prefixo}', target))

In [0]:
cols = restrito
modelos = [prefixo, *cols]

tab_KS = calcula_KS(modelo_lgbm, X_train, X_test, y_train, y_test, target, nscores=cols, restrito=restrito, prefixo=prefixo, safras=safras_dev, base_pd=base_cli_pd, base_cat=base_cli_cat)
tab_KS

vis.tabela_qs(modelos, "Safra_main", safras_dev)
vis.tabela_qs(modelos, "Safra_main", safras_nov)

#### Aberto

In [0]:
prefixo = "lgbm_aberto"
prefixos.append(prefixo)


otimizador = OtimizadorBayesiano(classificador="light_gbm", n_estimadores = 100, random_state=random_state, max_avaliacoes=100)
melhores_params, tentativas = otimizador.otimiza_hiperparams(X_aberto, y, tam_teste=.3)

In [0]:
modelo_lgbm_aberto = otimizador.modelo
target = 'inad30'

X_train, X_test, y_train, y_test = train_test_split(X_aberto, y, train_size=.7, random_state=random_state)
 
temp_train=X_train.join(y_train,rsuffix='r')
temp_train[f'yhat_{prefixo}']=modelo_lgbm_aberto.predict_proba(X_train[aberto])[:,1]
print("KS treino: " , calc_perf(temp_train,f'yhat_{prefixo}', target))
  
temp_test=X_test.join(y_test,rsuffix='r')
temp_test[f'yhat_{prefixo}']=modelo_lgbm_aberto.predict_proba(temp_test[aberto])[:,1]
print("KS teste: " , calc_perf(temp_test,f'yhat_{prefixo}', target))

In [0]:
cols = aberto
modelos = [prefixo, *cols]

tab_KS = calcula_KS(modelo_lgbm_aberto, X_train, X_test, y_train, y_test, target, nscores=cols, aberto=aberto, prefixo=prefixo, safras=safras_dev, base_pd=base_cli_pd, base_cat=base_cli_cat)
tab_KS

vis.tabela_qs(modelos, "Safra_main", safras_dev)
vis.tabela_qs(modelos, "Safra_main", safras_nov)

#### Aberto Contínuo

In [0]:
prefixo = "lgbm_aberto_continuo"
prefixos.append(prefixo)

otimizador = OtimizadorBayesiano(classificador="light_gbm", n_estimadores = 100, random_state=random_state, max_avaliacoes=100)
melhores_params, tentativas = otimizador.otimiza_hiperparams(X_aberto_continuo, y, tam_teste=.3)

In [0]:
modelo_lgbm_continuo = otimizador.modelo
target = 'inad30'
X_train, X_test, y_train, y_test = train_test_split(X_aberto_continuo, y, train_size=.7, random_state=random_state)

temp_train=X_train.join(y_train,rsuffix='r')
temp_train[f'yhat_{prefixo}']=modelo_lgbm_continuo.predict_proba(X_train[aberto_continuo])[:,1]
print("KS treino: " , calc_perf(temp_train,f'yhat_{prefixo}', target))
  
temp_test=X_test.join(y_test,rsuffix='r')
temp_test[f'yhat_{prefixo}']=modelo_lgbm_continuo.predict_proba(temp_test[aberto_continuo])[:,1]
print("KS teste: " , calc_perf(temp_test,f'yhat_{prefixo}', target))

In [0]:
cols = aberto_continuo
modelos = [prefixo, *cols]
 
tab_KS = calcula_KS(modelo_lgbm_continuo, X_train, X_test, y_train, y_test, target, nscores=cols, continuo=aberto_continuo, prefixo=prefixo, safras=safras_dev, base_pd=base_cli_pd, base_cat=base_cli_cat)
tab_KS
 
vis.tabela_qs(modelos, "Safra_main", safras_dev)
vis.tabela_qs(modelos, "Safra_main", safras_nov)

### Random Forest

#### Restrito

In [0]:
prefixo = "rf_restrito"
prefixos.append(prefixo)


otimizador = OtimizadorBayesiano(classificador="random_forest", n_estimadores = 100,  max_avaliacoes=100, random_state=random_state)
melhores_params, tentativas = otimizador.otimiza_hiperparams(X_restrito, y, tam_teste=.3)

In [0]:
modelo_rf = otimizador.modelo
target = 'inad30'

X_train, X_test, y_train, y_test = train_test_split(X_restrito, y, train_size=.7, random_state=random_state)
 
temp_train=X_train.join(y_train,rsuffix='r')
temp_train[f'yhat_{prefixo}']=modelo_rf.predict_proba(X_train[restrito])[:,1]
print("KS treino: " , calc_perf(temp_train,f'yhat_{prefixo}', target))
  
temp_test=X_test.join(y_test,rsuffix='r')
temp_test[f'yhat_{prefixo}']=modelo_rf.predict_proba(temp_test[restrito])[:,1]
print("KS teste: " , calc_perf(temp_test,f'yhat_{prefixo}', target))

In [0]:
cols = restrito
modelos = [prefixo, *cols]

tab_KS = calcula_KS(modelo_rf, X_train, X_test, y_train, y_test, target, nscores=cols, restrito=restrito, prefixo=prefixo, safras=safras_dev, base_pd=base_cli_pd, base_cat=base_cli_cat)
tab_KS

vis.tabela_qs(modelos, "Safra_main", safras_dev)
vis.tabela_qs(modelos, "Safra_main", safras_nov)

#### Aberto

In [0]:
prefixo = "rf_aberto"
prefixos.append(prefixo)


otimizador = OtimizadorBayesiano(classificador="random_forest", n_estimadores = 100, random_state=random_state, max_avaliacoes=100)
melhores_params, tentativas = otimizador.otimiza_hiperparams(X_aberto, y, tam_teste=.3)

In [0]:
modelo_rf_aberto = otimizador.modelo
target = 'inad30'

X_train, X_test, y_train, y_test = train_test_split(X_aberto, y, train_size=.7, random_state=random_state)
 
temp_train=X_train.join(y_train,rsuffix='r')
temp_train[f'yhat_{prefixo}']=modelo_rf_aberto.predict_proba(X_train[aberto])[:,1]
print("KS treino: " , calc_perf(temp_train,f'yhat_{prefixo}', target))
  
temp_test=X_test.join(y_test,rsuffix='r')
temp_test[f'yhat_{prefixo}']=modelo_rf_aberto.predict_proba(temp_test[aberto])[:,1]
print("KS teste: " , calc_perf(temp_test,f'yhat_{prefixo}', target))

In [0]:
cols = aberto
modelos = [prefixo, *cols]

tab_KS = calcula_KS(modelo_rf_aberto, X_train, X_test, y_train, y_test, target, nscores=cols, aberto=aberto, prefixo=prefixo, safras=safras_dev, base_pd=base_cli_pd, base_cat=base_cli_cat)
tab_KS

vis.tabela_qs(modelos, "Safra_main", safras_dev)
vis.tabela_qs(modelos, "Safra_main", safras_nov)

#### Aberto Contínuo

In [0]:
prefixo = "rf_aberto_continuo"
prefixos.append(prefixo)

otimizador = OtimizadorBayesiano(classificador="random_forest", n_estimadores = 100, random_state=random_state, max_avaliacoes=100)
melhores_params, tentativas = otimizador.otimiza_hiperparams(X_aberto_continuo, y, tam_teste=.3)

In [0]:
modelo_rf_continuo = otimizador.modelo
target = 'inad30'
X_train, X_test, y_train, y_test = train_test_split(X_aberto_continuo, y, train_size=.7, random_state=random_state)

temp_train=X_train.join(y_train,rsuffix='r')
temp_train[f'yhat_{prefixo}']=modelo_rf_continuo.predict_proba(X_train[aberto_continuo])[:,1]
print("KS treino: " , calc_perf(temp_train,f'yhat_{prefixo}', target))
  
temp_test=X_test.join(y_test,rsuffix='r')
temp_test[f'yhat_{prefixo}']=modelo_rf_continuo.predict_proba(temp_test[aberto_continuo])[:,1]
print("KS teste: " , calc_perf(temp_test,f'yhat_{prefixo}', target))

In [0]:
cols = aberto_continuo
modelos = [prefixo, *cols]

tab_KS = calcula_KS(modelo_rf_continuo, X_train, X_test, y_train, y_test, target, nscores=cols, continuo=aberto_continuo, prefixo=prefixo, safras=safras_dev, base_pd=base_cli_pd, base_cat=base_cli_cat)
tab_KS

vis.tabela_qs(modelos, "Safra_main", safras_dev)
vis.tabela_qs(modelos, "Safra_main", safras_nov)

### Validação

In [0]:
target = 'inad30'
modelo_logistica = "yhat_blend"

cols0 = [prefixo for prefixo in prefixos]
cols = [*cols0, modelo_logistica, 'P4_BVS','SCORE_HSPN','QuodScore_v2']
safras_ks = [safra for safra in pd.DatetimeIndex(base_cli_pd["Safra_main"].unique()) if safra.year < 2022 and safra.month <= 12]

vis.tabela_ks_por_safra(target, modelos=cols, safras=safras_ks)

### Gráficos

##### Ks por safra

In [0]:
graficos = vis.ks_por_safra(cols, safras_dev)

In [0]:
grafico_ord = vis.ordenacao_inad_decil(cols)

In [0]:
restritos = [modelo for modelo in cols if "restrito" in modelo]
abertos = [modelo for modelo in cols if "aberto" in modelo and "continuo" not in modelo]
continuos = [modelo for modelo in cols if "continuo" in modelo]

restritos_blend = [modelo_logistica, *restritos]
abertos_blend = [modelo_logistica, *abertos]
continuos_blend = [modelo_logistica, *continuos]

In [0]:
vis.tab_KS_pivot[restritos_blend].plot(kind="line", title="KS do modelo por safra (Restrito)", figsize=(8,6), marker='.', markersize=14)
plt.ylabel("KS")
plt.xlabel("Safra - 2021")

In [0]:
vis.tab_KS_pivot[abertos_blend].plot.line(title="KS do modelo por safra (Aberto)", figsize=(8,6), marker='.', markersize=14)
plt.ylabel("KS")
plt.xlabel("Safra - 2021")

In [0]:
vis.tab_KS_pivot[continuos_blend].plot.line(title="KS do modelo  por safra (Contínuo)", figsize=(8,6), marker='.', markersize=14)
plt.ylabel("KS")
plt.xlabel("Safra - 2021")

In [0]:
foras = [modelo for modelo in restritos_blend if "ab" in modelo or "rf" in modelo]
atualizados = [modelo for modelo in restritos_blend if modelo not in foras]
vis.tab_KS_pivot[atualizados].plot.line(title="KS de cada modelo por safra", figsize=(10,8), marker='.', markersize=14, xlim=[safras_dev[0], safras_dev[-1]], ylim=[40,])\

In [0]:
foras = [modelo for modelo in abertos_blend if "yhat_ab" in modelo]
atualizados = [modelo for modelo in abertos_blend if modelo not in foras]
vis.tab_KS_pivot[atualizados].plot.line(title="KS de cada modelo por safra", figsize=(10,8), marker='.', markersize=14, xlim=[safras_dev[0], safras_dev[-1]], ylim=[20,])\

In [0]:
foras = [modelo for modelo in continuos_blend if "yhat_ab" in modelo or "yhat_lgbm" in modelo or "yhat_rf" in modelo]
atualizados = [modelo for modelo in continuos_blend if modelo not in foras]

vis.tab_KS_pivot[atualizados].plot.line(title="KS de cada modelo por safra", figsize=(10,10), marker='.', markersize=14, xlim=[safras_dev[0], safras_dev[-1]], ylim=[20,])\

In [0]:
vis.tab_KS_pivot[continuos_blend].plot.line(title="KS de cada modelo por safra", figsize=(10,8), marker='.', markersize=14, xlim=[safras_dev[0], safras_dev[-1]], ylim=[40,])\

In [0]:
foras_aberto = [modelo for modelo in abertos_blend if "yhat_ab" in modelo]
atualizados_aberto = [modelo for modelo in abertos_blend if modelo not in foras_aberto]

foras_restrito = [modelo for modelo in restritos_blend if "ab" in modelo or "rf" in modelo]
atualizados_restrito = [modelo for modelo in restritos_blend if modelo not in foras]


atualizados_final = [modelo_logistica, "yhat_xg_restrito", "yhat_lgbm_restrito", "yhat_xg_aberto_continuo"]

vis.tab_KS_pivot[atualizados_final].plot.line(title="KS de cada modelo por safra", figsize=(10,6), marker='.', markersize=14, xlim=[safras_dev[0], safras_dev[-1]], ylim=[20,])\

#### Decis Inad

In [0]:
prefixos = list(dict.fromkeys(prefixos))
 
scores = [modelo_logistica, 'P4_BVS', 'SCORE_HSPN', 'QuodScore_v2']

tab_decis = vis.tabela_qs_final(prefixos, scores)
tab_decis

In [0]:
tab_decis[restritos_blend].plot(kind="line", title="Ordenação inad por decil (Restrito)", figsize=(8,6), marker='.', markersize=14 )
plt.xlabel("Decil")
plt.ylabel("%inad30")
plt.legend(title="modelo")

In [0]:
tab_decis[abertos_blend].plot(kind="line", title="Ordenação inad por decil (Aberto)", figsize=(8,6), marker='.', markersize=14 )
plt.xlabel("Decil")
plt.ylabel("%inad30")
plt.legend(title="modelo")

In [0]:
tab_decis[continuos_blend].plot(kind="line", title="Ordenação inad por decil (Contínuo)", figsize=(8,6), marker='.', markersize=14 )
plt.xlabel("Decil")
plt.ylabel("%inad30")
plt.legend(title="modelo")

In [0]:
atualizados = [modelo_logistica, "yhat_ab_aberto", "yhat_ab_restrito", "yhat_ab_aberto_continuo"]
tab_decis[atualizados].plot.line(title="Ordenação inad por decil", figsize=(10,6), marker='.', markersize=14, xlim=[safras_dev[0], safras_dev[-1]], ylim=[20,])\